In [1]:
# This is the import cell
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)
import matplotlib.ticker as ticker
from collections import OrderedDict
from scipy import stats
from PIL import Image
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import pylab as plot
first = True
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 10)



# Here are my rc parameters for matplotlibf
fsize = 20
mpl.rc('font', serif='Helvetica Neue')
#mpl.rc('font', serif='Times New Roman')
mpl.rcParams.update({'font.size': fsize})
mpl.rcParams['figure.figsize'] = 3.2, 2.8
mpl.rcParams['figure.dpi'] = 500
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['axes.linewidth'] = 1.5
# Set x tick params
mpl.rcParams['xtick.major.size'] = 4.5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['xtick.minor.size'] = 3.
mpl.rcParams['xtick.minor.width'] = 1.25
# Set y tick params
mpl.rcParams['ytick.major.size'] = 4.5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['ytick.minor.size'] = 3.
mpl.rcParams['ytick.minor.width'] = 1.25
mpl.rcParams['legend.fontsize']= 20.
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams["font.family"] = "Times New Roman"
# Load LaTeX and amsmath
# mpl.rc('text', usetex=True)
# mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

In [2]:
# Get the current path
if first:
    parent = os.getcwd()
print(parent)
# Grab file names from data folder
dens = os.listdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/Align_press_CoM')
try:
    dens.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
    
# Grab the MCS data
pres = os.listdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/Interpart_press')
try:
    pres.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
    
# Grab the MCS data
pres2 = os.listdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/BubComp2')
try:
    pres2.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
# Grab the MCS data
pres3 = os.listdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/PhaseComp2')
try:
    pres3.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
# Grab the MCS data
pres4 = os.listdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/radial')
try:
    pres4.remove('.DS_Store')
except:
    print(".DS_Store not in directory")    
    
# Grab the MCS data
pres5 = os.listdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/lat')
try:
    pres5.remove('.DS_Store')
except:
    print(".DS_Store not in directory")    
    

/Users/nicklauersdorf/hoomd-blue/build/njlauersdorf/ABPs/post_proc/Jupyter
.DS_Store not in directory
.DS_Store not in directory
.DS_Store not in directory
.DS_Store not in directory
.DS_Store not in directory
.DS_Store not in directory


In [3]:
# Some functions to get the relevant data from the filenames
def checkFile(fname, string):
    for i in range(len(fname)):
        if fname[i] == string[0]:
#             print"{} matches {}".format(fname[i], string[0])
            for j in range(1, len(string)):
                if (i + j) > (len(fname) - 1):
                    break
                elif fname[i + j] == string[j]:
#                     print"{} matches {}".format(fname[i+j], string[j])
                    if j == (len(string) - 1):
#                         print"Final match!"
                        return True
                else:
                    break
    return False
    
def txtValue(fname, string):
    out = ""
    index = 0
    for i in range(len(fname)):
        if fname[i] == string[0]:
            for j in range(1, len(string)):
                if (i + j) > (len(fname) - 1):
                    break
                elif fname[i + j] == string[j]:
                    if j == (len(string) - 1):
                        # Last index of search string
                        index = i + j
                else:
                    break
                        
    # First index of value
    index += 1
    mybool = True
    while mybool:
        if fname[index].isdigit():
            out = out + fname[index]
            index += 1
        elif fname[index] == ".":    
            if fname[index+1].isdigit():
                out = out + fname[index]
                index += 1
            else:
                mybool = False
        else:
            mybool = False
    return float(out)

# Sorting functions
def multiSort(arr1, arr2, arr3, arr4):
    """Sort an array the slow (but certain) way, returns original indices in sorted order"""
    # Doing this for PeR, PeS, xS in this case
    cpy1 = np.copy(arr1)
    cpy2 = np.copy(arr2)
    cpy3 = np.copy(arr3)
    cpy4 = np.copy(arr4)
    ind = np.arange(0, len(arr1))
    for i in range(len(cpy1)):
        for j in range(len(cpy1)):
            # Sort by first variable
            if cpy1[i] > cpy1[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
                
            # If first variable is equal, resort to second variable
            elif cpy1[i] == cpy1[j] and cpy2[i] > cpy2[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
                
            elif cpy1[i] == cpy1[j] and cpy2[i] == cpy2[j] and cpy3[i] > cpy3[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
            elif cpy1[i] == cpy1[j] and cpy2[i] == cpy2[j] and cpy3[i] == cpy3[j] and cpy4[i] > cpy4[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
    return ind

def indSort(arr1, arr2):
    """Take sorted index array, use to sort array"""
    # arr1 is array to sort
    # arr2 is index array
    cpy = np.copy(arr1)
    for i in range(len(arr1)):
        arr1[i] = cpy[arr2[i]]

In [4]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in dens:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 
                
# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(dens, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_dens = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/Align_press_CoM')
for i in dens:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_dens.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_dens[0])

,tauB,sizeBin,clust_size,press_align,press_alignA,press_alignB
0,0.0,5.016308,28090,550.336010,-68.538778,618.874788
1,0.1,5.016308,28567,491.638569,-111.354454,602.993022
2,0.2,5.016308,29079,473.132525,-92.591965,565.724490
3,0.3,5.016308,29520,481.344598,-38.731670,520.076268
4,0.4,5.016308,30023,495.445240,-24.650867,520.096107
...,...,...,...,...,...,...
1994,199.4,5.016308,35415,77.762958,-19.642633,97.405591
1995,199.5,5.016308,35534,107.216612,-43.948668,151.165279
1996,199.6,5.016308,35470,175.709522,-34.452537,210.162059
1997,199.7,5.016308,35439,252.960096,30.736209,222.223887


In [5]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params = pd.DataFrame(columns=headers)
for i in range(0, len(all_dens)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params = params.append(df, ignore_index = True)
display(params)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,100,100,50.0,1.0,60.0,0.000001
1,0,150,100,50.0,1.0,60.0,0.000001
2,0,150,150,50.0,1.0,60.0,0.000001
3,0,200,100,50.0,1.0,60.0,0.000001
4,0,200,150,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
21,0,450,0,50.0,1.0,60.0,0.000001
22,0,450,100,50.0,1.0,60.0,0.000001
23,0,450,200,50.0,1.0,60.0,0.000001
24,0,450,250,50.0,1.0,60.0,0.000001


In [6]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_dens)):
    # Ger rid of NaN in favor of 0
    all_dens[i].fillna(0, inplace=True)

headers=list(all_dens[0])
display(all_dens[0])
print(all_dens[0][headers[1]][0])
print(all_dens[0][headers[2]][0])

,tauB,sizeBin,clust_size,press_align,press_alignA,press_alignB
0,0.0,5.016308,28090,550.336010,-68.538778,618.874788
1,0.1,5.016308,28567,491.638569,-111.354454,602.993022
2,0.2,5.016308,29079,473.132525,-92.591965,565.724490
3,0.3,5.016308,29520,481.344598,-38.731670,520.076268
4,0.4,5.016308,30023,495.445240,-24.650867,520.096107
...,...,...,...,...,...,...
1994,199.4,5.016308,35415,77.762958,-19.642633,97.405591
1995,199.5,5.016308,35534,107.216612,-43.948668,151.165279
1996,199.6,5.016308,35470,175.709522,-34.452537,210.162059
1997,199.7,5.016308,35439,252.960096,30.736209,222.223887


5.016308
28090


In [7]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/Interpart_press')
for i in pres:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres[-1])

,Timestep,gasArea,gasSigXX,gasSigXY,gasSigYX,gasSigYY,gasTrace,bulkArea,bulkSigXX,bulkSigXY,bulkSigYX,bulkSigYY,bulkTrace,Length,NDense
0,0.0,38527.643,12339.243,-154.004,-154.004,11981.698,12006.466,24574.979,3.519521e+07,10647.549,10647.549,3.519747e+07,3.520699e+07,531.9,41134
1,0.1,36618.599,23280.374,-654.134,-654.134,21288.168,21630.137,22292.509,4.671091e+07,-336233.582,-336233.582,4.726104e+07,4.664974e+07,1523.7,41061
2,0.2,35940.336,72813.196,-482.231,-482.231,78778.334,75313.534,20698.463,5.269610e+07,-445154.951,-445154.951,5.302322e+07,5.241451e+07,2114.3,40823
3,0.3,37428.958,875747.832,1073.566,1073.566,874727.092,876311.028,19846.188,5.763997e+07,-364879.246,-364879.246,5.974005e+07,5.832513e+07,1910.3,26063
4,0.4,36383.620,210494.356,3101.538,3101.538,211247.726,213972.579,20627.334,5.899276e+07,-190155.183,-190155.183,5.943629e+07,5.902437e+07,1913.7,28834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,199.5,48485.667,2164490.527,-13735.963,-13735.963,2187777.193,2162397.896,10856.024,5.074735e+07,153944.795,153944.795,5.042688e+07,5.074106e+07,1683.8,23764
1996,199.6,48829.879,2281648.235,-14629.139,-14629.139,2261462.735,2256926.346,11024.954,5.038507e+07,-606611.356,-606611.356,4.915761e+07,4.916473e+07,1521.5,23580
1997,199.7,48481.856,2349650.710,8414.601,8414.601,2340249.266,2353364.589,10805.218,4.960720e+07,-794804.963,-794804.963,4.899458e+07,4.850609e+07,1715.3,23750
1998,199.8,47761.678,2227398.804,12810.533,12810.533,2240059.979,2246539.925,10880.157,4.937122e+07,-991418.937,-991418.937,4.876501e+07,4.807670e+07,1932.8,24106


In [8]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params2 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params2 = params2.append(df, ignore_index = True)
display(params2)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,100,100,50.0,1.0,60.0,0.000001
1,0,150,100,50.0,1.0,60.0,0.000001
2,0,150,150,50.0,1.0,60.0,0.000001
3,0,200,100,50.0,1.0,60.0,0.000001
4,0,200,150,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
21,0,450,250,50.0,1.0,60.0,0.000001
22,0,450,350,50.0,1.0,60.0,0.000001
23,0,450,450,50.0,1.0,60.0,0.000001
24,0,450,50,50.0,1.0,60.0,0.000001


In [9]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres)):
    # Ger rid of NaN in favor of 0
    all_pres[i].fillna(0, inplace=True)

headers=list(all_pres[0])
display(all_pres[0])
print(all_pres[0][headers[1]][0])
print(all_pres[0][headers[2]][0])

,Timestep,gasArea,gasSigXX,gasSigXY,gasSigYX,gasSigYY,gasTrace,bulkArea,bulkSigXX,bulkSigXY,bulkSigYX,bulkSigYY,bulkTrace,Length,NDense
0,0.0,44841.589,90354.471,1306.579,1306.579,89874.079,91420.854,18433.775,1.162850e+07,-473.247,-473.247,1.162747e+07,1.162751e+07,489.1,28090
1,0.1,44959.714,833984.872,-1194.812,-1194.812,836546.354,834070.801,18201.336,1.482193e+07,1516.362,1516.362,1.497799e+07,1.490148e+07,520.7,28567
2,0.2,44614.231,821227.344,2402.590,2402.590,819091.428,822561.976,18506.174,1.553521e+07,98215.598,98215.598,1.556530e+07,1.564847e+07,535.3,29079
3,0.3,44225.563,775778.057,1274.883,1274.883,765483.677,771905.751,18800.850,1.583556e+07,121404.865,121404.865,1.574416e+07,1.591127e+07,569.1,29520
4,0.4,43734.013,768738.642,-2527.938,-2527.938,768711.139,766196.953,19180.626,1.585247e+07,12972.694,12972.694,1.578959e+07,1.583400e+07,604.1,30023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,199.5,38400.628,393099.661,-822.650,-822.650,398043.908,394749.134,23059.683,1.658549e+07,-33769.563,-33769.563,1.656189e+07,1.653992e+07,969.2,35534
1996,199.6,38380.305,386029.159,-552.611,-552.611,393161.621,389042.778,22903.454,1.645418e+07,-22424.495,-22424.495,1.647558e+07,1.644246e+07,1012.1,35470
1997,199.7,38523.833,390047.656,-1987.835,-1987.835,396376.662,391224.324,22919.966,1.639145e+07,-40421.206,-40421.206,1.626144e+07,1.628602e+07,965.8,35439
1998,199.8,38340.930,396028.178,-1271.395,-1271.395,391523.926,392504.657,23007.607,1.639467e+07,-93504.386,-93504.386,1.631133e+07,1.625950e+07,990.6,35536


44841.58900000001
90354.47099999999


In [10]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres2:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres2, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres_new = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/BubComp2')
for i in pres2:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres_new.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres_new[-1])
print(storeVals)

,tauB,sizeBin,clust_size,int_id,bub_id,Na,Nb,radius,radius_err,sa_ext,sa_int,edge_width,edge_width_err,edge_begin,edge_end,NBin
0,0.0,5.016308,42651,1,1,6647,18354,94.161009,1.216715,611.180966,399.659987,32.266163,1.788011,61.894846,94.161009,753
1,0.1,5.016308,42696,1,1,6380,18145,91.591220,2.384943,594.909385,433.342241,30.377556,3.324280,61.213664,91.591220,701
2,0.1,5.016308,42696,1,4,369,0,5.302671,2.221200,40.523849,0.000000,0.000000,0.000000,0.000000,0.000000,11
3,0.1,5.016308,42696,1,5,369,0,4.642710,1.422925,37.589696,0.000000,0.000000,0.000000,0.000000,0.000000,11
4,0.2,5.016308,42807,1,1,5362,18450,97.859355,32.741957,607.415514,83.439809,89.119730,33.289572,8.739625,97.859355,703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,199.5,5.016308,31828,1,1,11813,8606,83.746766,15.861620,595.269030,384.665330,30.643629,6.483743,53.103138,83.746766,636
2236,199.6,5.016308,31693,1,1,12023,8621,84.022612,17.768463,594.599746,370.996107,32.486913,9.187860,51.535699,84.022612,635
2237,199.6,5.016308,31693,1,2,35,151,7.163674,1.207087,48.576151,0.000000,0.000000,0.000000,0.000000,0.000000,16
2238,199.7,5.016308,31530,2,2,11595,8327,83.493161,16.110552,599.929397,378.423927,30.406892,6.823248,53.086269,83.493161,624


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [100.0, 150.0, 150.0, 200.0, 200.0, 200.0, 200.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 350.0, 350.0, 350.0, 350.0, 350.0, 350.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0], [100.0, 100.0, 150.0, 100.0, 150.0, 200.0, 50.0, 50.0, 150.0, 200.0, 250.0, 0.0, 100.0, 250.0, 350.0, 50.0, 0.0, 100.0, 200.0, 200.0, 250.0, 350.0, 450.0, 50.0, 0.0, 100.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params3 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres_new)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params3 = params3.append(df, ignore_index = True)
display(params3)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,100,100,50.0,1.0,60.0,0.000001
1,0,150,100,50.0,1.0,60.0,0.000001
2,0,150,150,50.0,1.0,60.0,0.000001
3,0,200,100,50.0,1.0,60.0,0.000001
4,0,200,150,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
21,0,450,350,50.0,1.0,60.0,0.000001
22,0,450,450,50.0,1.0,60.0,0.000001
23,0,450,50,50.0,1.0,60.0,0.000001
24,0,450,0,50.0,1.0,60.0,0.000001


In [12]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres_new)):
    # Ger rid of NaN in favor of 0
    all_pres_new[i].fillna(0, inplace=True)

headers=list(all_pres_new[0])
display(all_pres_new[0])
print(all_pres_new[0][headers[1]][0])
print(all_pres_new[0][headers[2]][0])

,tauB,sizeBin,clust_size,int_id,bub_id,Na,Nb,radius,radius_err,sa_ext,sa_int,edge_width,edge_width_err,edge_begin,edge_end,NBin
0,0.0,5.016308,28090,1,1,8590,5577,85.464927,2.475714,567.574263,419.577458,22.521887,3.940705,62.943040,85.464927,541
1,0.1,5.016308,28567,1,1,7896,5484,84.841898,3.089875,573.253602,419.907640,20.992721,3.568674,63.849177,84.841898,519
2,0.2,5.016308,29079,1,1,7352,6089,86.468648,2.912976,612.014539,421.047985,21.857546,3.093040,64.611103,86.468648,544
3,0.2,5.016308,29079,1,2,0,163,5.104896,1.371347,36.832149,0.000000,0.000000,0.000000,0.000000,0.000000,12
4,0.3,5.016308,29520,1,1,6683,6397,86.960384,2.952896,604.417926,429.886452,21.292444,3.352464,65.667941,86.960384,538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,199.7,5.016308,35439,2,2,11405,11714,98.402149,9.453593,277.299556,767.574846,36.978586,27.545110,62.162622,98.402149,919
6119,199.7,5.016308,35439,2,1,166,196,75.414789,40.021811,605.999350,0.000000,0.000000,0.000000,0.000000,0.000000,39
6120,199.7,5.016308,35439,2,3,131,166,12.902172,5.210540,104.858109,0.000000,0.000000,0.000000,0.000000,0.000000,30
6121,199.7,5.016308,35439,2,4,57,70,6.960229,2.741544,51.329150,0.000000,0.000000,0.000000,0.000000,0.000000,14


5.016308
28090


In [13]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres3:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres3, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres_new2 = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/PhaseComp2')
for i in pres3:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres_new2.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres_new2[-1])
print(storeVals)

,tauB,sizeBin,clust_size,Na_bulk,Nb_bulk,NBin_bulk,Na_gas,Nb_gas,NBin_gas,Na_int,Nb_int,NBin_int,Na_bub,Nb_bub,NBin_bub
0,0.0,5.016308,41134,17963,0,434,0,8263,1453,6628,16764,704,382,0,10
1,0.1,5.016308,41061,15015,8,360,101,7667,1354,9557,17352,877,300,0,10
2,0.2,5.016308,40823,5413,0,119,269,6876,1225,19291,18151,1257,0,0,0
3,0.3,5.016308,26063,2469,43,55,798,6026,1150,21706,18958,1396,0,0,0
4,0.4,5.016308,28834,2590,4507,175,1388,5334,1032,20995,15186,1394,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,199.4,5.016308,24116,4544,1993,133,8615,18184,1994,11814,4850,474,0,0,0
1995,199.5,5.016308,23764,4300,1901,126,8572,17908,1971,12101,5218,504,0,0,0
1996,199.6,5.016308,23580,4536,2009,134,8491,17461,1928,11946,5557,539,0,0,0
1997,199.7,5.016308,23750,4415,1931,130,8569,17293,1898,11907,5717,551,82,86,22


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [100.0, 150.0, 150.0, 200.0, 200.0, 200.0, 200.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 350.0, 350.0, 350.0, 350.0, 350.0, 350.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0], [100.0, 100.0, 150.0, 100.0, 150.0, 200.0, 50.0, 200.0, 250.0, 50.0, 100.0, 150.0, 0.0, 200.0, 250.0, 350.0, 50.0, 0.0, 100.0, 100.0, 200.0, 250.0, 350.0, 450.0, 50.0, 0.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params4 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres_new)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params4 = params4.append(df, ignore_index = True)
display(params4)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,100,100,50.0,1.0,60.0,0.000001
1,0,150,100,50.0,1.0,60.0,0.000001
2,0,150,150,50.0,1.0,60.0,0.000001
3,0,200,100,50.0,1.0,60.0,0.000001
4,0,200,150,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
21,0,450,250,50.0,1.0,60.0,0.000001
22,0,450,350,50.0,1.0,60.0,0.000001
23,0,450,450,50.0,1.0,60.0,0.000001
24,0,450,50,50.0,1.0,60.0,0.000001


In [15]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres_new2)):
    # Ger rid of NaN in favor of 0
    all_pres_new2[i].fillna(0, inplace=True)

headers=list(all_pres_new2[0])
display(all_pres_new2[0])
print(all_pres_new2[0][headers[1]][0])
print(all_pres_new2[0][headers[2]][0])

,tauB,sizeBin,clust_size,Na_bulk,Nb_bulk,NBin_bulk,Na_gas,Nb_gas,NBin_gas,Na_int,Nb_int,NBin_int,Na_bub,Nb_bub,NBin_bub
0,0.0,5.016308,28090,16383,0,445,0,19450,1615,8590,5577,541,0,0,0
1,0.1,5.016308,28567,17077,0,458,0,19543,1624,7896,5484,519,0,0,0
2,0.2,5.016308,29079,17621,0,470,0,18775,1575,7352,6089,544,0,163,12
3,0.3,5.016308,29520,18290,0,486,0,18630,1577,6683,6397,538,0,0,0
4,0.4,5.016308,30023,19223,0,512,0,17841,1529,5750,7186,560,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,199.4,5.016308,35415,7444,6397,374,5436,6577,1313,11966,11931,887,127,122,27
1995,199.5,5.016308,35534,8856,7574,445,5703,6898,1371,10367,10491,773,47,64,12
1996,199.6,5.016308,35470,8332,7165,419,5590,6716,1342,11051,11146,840,0,0,0
1997,199.7,5.016308,35439,8235,6946,411,4979,5935,1188,11405,11714,919,354,432,83


5.016308
28090


In [16]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres4:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres4, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres_new3 = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/radial')
for i in pres4:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres_new3.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres_new3[-1])
print(storeVals)

,tauB,sizeBin,clust_size,interface_id,bub_id,r_min,r_max,align,alignA,alignB,num_dens,num_densA,num_densB,press,pressA,pressB
0,0.0,5.016308,2,1,0,0.000000,3.045615,-0.059889,-0.059889,NaN,1.887395,1.887395,0.000000,-50.865574,-50.865574,0.000000
1,0.0,5.016308,2,1,0,3.045615,6.091230,-0.009640,-0.009640,NaN,1.715813,1.715813,0.000000,-7.443101,-7.443101,0.000000
2,0.0,5.016308,2,1,0,6.091230,9.136846,0.033282,0.033282,NaN,1.777583,1.777583,0.000000,26.622637,26.622637,0.000000
3,0.0,5.016308,2,1,0,9.136846,12.182461,-0.009756,-0.009756,NaN,1.799153,1.799153,0.000000,-7.898621,-7.898621,0.000000
4,0.0,5.016308,2,1,0,12.182461,15.228076,-0.003113,-0.003113,NaN,1.731065,1.731065,0.000000,-2.425138,-2.425138,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83953,199.8,5.016308,9,1,0,112.687764,115.733379,0.005355,-0.014306,0.019108,0.150076,0.061769,0.088307,0.192941,-0.397643,0.590584
83954,199.8,5.016308,9,1,0,115.733379,118.778994,-0.023963,-0.047760,-0.005686,0.152863,0.066404,0.086459,-1.599229,-1.427157,-0.172072
83955,199.8,5.016308,9,1,0,118.778994,121.824610,0.002328,-0.002541,0.006588,0.156378,0.072976,0.083402,0.108850,-0.083454,0.192305
83956,199.8,5.016308,9,1,0,121.824610,124.870225,-0.063944,-0.044073,-0.077738,0.147857,0.060583,0.087273,-3.576115,-1.201543,-2.374573


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [100.0, 150.0, 150.0, 200.0, 200.0, 200.0, 200.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 350.0, 350.0, 350.0, 350.0, 350.0, 350.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0], [100.0, 100.0, 150.0, 100.0, 150.0, 200.0, 50.0, 50.0, 0.0, 100.0, 150.0, 200.0, 250.0, 0.0, 100.0, 200.0, 250.0, 350.0, 50.0, 450.0, 50.0, 0.0, 100.0, 200.0, 250.0, 350.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params5 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres_new3)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params5 = params5.append(df, ignore_index = True)
display(params5)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,100,100,50.0,1.0,60.0,0.000001
1,0,150,100,50.0,1.0,60.0,0.000001
2,0,150,150,50.0,1.0,60.0,0.000001
3,0,200,100,50.0,1.0,60.0,0.000001
4,0,200,150,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
21,0,450,0,50.0,1.0,60.0,0.000001
22,0,450,100,50.0,1.0,60.0,0.000001
23,0,450,200,50.0,1.0,60.0,0.000001
24,0,450,250,50.0,1.0,60.0,0.000001


In [18]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres_new3)):
    # Ger rid of NaN in favor of 0
    all_pres_new3[i].fillna(0, inplace=True)

headers=list(all_pres_new3[0])
display(all_pres_new3[0])
print(all_pres_new3[0][headers[1]][0])
print(all_pres_new3[0][headers[2]][0])

,tauB,sizeBin,clust_size,interface_id,bub_id,r_min,r_max,align,alignA,alignB,num_dens,num_densA,num_densB,press,pressA,pressB
0,0.0,5.016308,3,1,0,0.000000,3.045615,-0.050720,-0.050720,0.000000,1.269702,1.269702,0.000000,-6.439869,-6.439869,0.000000
1,0.0,5.016308,3,1,0,3.045615,6.091230,-0.036610,-0.036610,0.000000,1.452722,1.452722,0.000000,-5.318469,-5.318469,0.000000
2,0.0,5.016308,3,1,0,6.091230,9.136846,0.042313,0.042313,0.000000,1.475600,1.475600,0.000000,6.243758,6.243758,0.000000
3,0.0,5.016308,3,1,0,9.136846,12.182461,-0.003304,-0.003304,0.000000,1.460893,1.460893,0.000000,-0.482664,-0.482664,0.000000
4,0.0,5.016308,3,1,0,12.182461,15.228076,-0.020111,-0.020111,0.000000,1.475600,1.475600,0.000000,-2.967511,-2.967511,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83953,199.8,5.016308,21,1,0,112.687764,115.733379,0.002684,-0.045101,0.043644,0.344993,0.159227,0.185765,0.092611,-0.718136,0.810748
83954,199.8,5.016308,21,1,0,115.733379,118.778994,0.064446,-0.013896,0.135495,0.356087,0.169353,0.186734,2.294822,-0.235327,2.530149
83955,199.8,5.016308,21,1,0,118.778994,121.824610,0.014304,-0.035752,0.059573,0.380520,0.180703,0.199816,0.544313,-0.646051,1.190364
83956,199.8,5.016308,21,1,0,121.824610,124.870225,0.005073,-0.004954,0.013790,0.351636,0.163532,0.188104,0.178391,-0.081010,0.259401


5.016308
3


In [19]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres5:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres5, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres_new4 = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/slow_out_txt_files/lat')
for i in pres5:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres_new4.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres_new4[-1])
print(storeVals)

,tauB,sizeBin,clust_size,lat_mean_bulk,lat_mean_int,lat_mean_bub,lat_mean_all,lat_std_bulk,lat_std_int,lat_std_bub,lat_std_all
0,0.0,5.016308,44899,0.806966,0.858114,0.80699,0.842690,0.002438,0.217425,0.002446,0.183222
1,0.1,5.016308,45162,0.707837,0.764502,0.00000,0.742579,0.007224,0.214102,0.000000,0.169961
2,0.2,5.016308,45556,0.709414,0.778611,0.00000,0.752290,0.004863,0.221807,0.000000,0.177825
3,0.3,5.016308,45818,0.720120,0.793412,0.00000,0.762968,0.004948,0.219350,0.000000,0.171591
4,0.4,5.016308,45902,0.731635,0.803361,0.00000,0.771615,0.005031,0.210728,0.000000,0.161347
...,...,...,...,...,...,...,...,...,...,...,...
1994,199.4,5.016308,42977,0.770186,0.866280,0.00000,0.809662,0.010821,0.292635,0.000000,0.193607
1995,199.5,5.016308,43231,0.771058,0.859453,0.00000,0.806957,0.009904,0.262691,0.000000,0.173113
1996,199.6,5.016308,43401,0.772302,0.871247,0.00000,0.811917,0.010340,0.300763,0.000000,0.196549
1997,199.7,5.016308,43428,0.773350,0.865825,0.00000,0.810648,0.010553,0.283268,0.000000,0.185710


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [100.0, 150.0, 150.0, 200.0, 200.0, 200.0, 200.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 350.0, 350.0, 350.0, 350.0, 350.0, 350.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0, 450.0], [100.0, 100.0, 150.0, 100.0, 150.0, 200.0, 50.0, 50.0, 0.0, 100.0, 150.0, 200.0, 250.0, 0.0, 100.0, 200.0, 250.0, 350.0, 50.0, 450.0, 50.0, 0.0, 100.0, 200.0, 250.0, 350.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params6 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres_new4)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params6 = params6.append(df, ignore_index = True)
display(params6)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,100,100,50.0,1.0,60.0,0.000001
1,0,150,100,50.0,1.0,60.0,0.000001
2,0,150,150,50.0,1.0,60.0,0.000001
3,0,200,100,50.0,1.0,60.0,0.000001
4,0,200,150,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
21,0,450,0,50.0,1.0,60.0,0.000001
22,0,450,100,50.0,1.0,60.0,0.000001
23,0,450,200,50.0,1.0,60.0,0.000001
24,0,450,250,50.0,1.0,60.0,0.000001


In [21]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres_new4)):
    # Ger rid of NaN in favor of 0
    all_pres_new4[i].fillna(0, inplace=True)

headers=list(all_pres_new4[0])
display(all_pres_new4[0])
print(all_pres_new4[0][headers[1]][0])
print(all_pres_new4[0][headers[2]][0])

,tauB,sizeBin,clust_size,lat_mean_bulk,lat_mean_int,lat_mean_bub,lat_mean_all,lat_std_bulk,lat_std_int,lat_std_bub,lat_std_all
0,0.0,5.016308,28090,0.888678,1.030912,0.000000,0.954637,0.004893,0.257686,0.000000,0.189306
1,0.1,5.016308,28567,0.884760,1.003274,0.000000,0.936825,0.012394,0.281075,0.000000,0.195582
2,0.2,5.016308,29079,0.882378,1.018494,1.415733,0.943754,0.015054,0.303376,0.241022,0.213875
3,0.3,5.016308,29520,0.880835,1.024554,0.000000,0.940760,0.015585,0.307165,0.000000,0.210958
4,0.4,5.016308,30023,0.881377,1.049716,0.000000,0.949092,0.015991,0.324939,0.000000,0.222348
...,...,...,...,...,...,...,...,...,...,...,...
1994,199.4,5.016308,35415,0.890208,1.004265,1.663589,0.967029,0.022836,0.245287,0.373040,0.212496
1995,199.5,5.016308,35534,0.891513,1.003015,1.658195,0.955975,0.023932,0.238028,0.373091,0.191798
1996,199.6,5.016308,35470,0.890506,1.011309,0.000000,0.961644,0.022785,0.248381,0.000000,0.200190
1997,199.7,5.016308,35439,0.891139,1.026354,1.644311,0.986263,0.022422,0.269601,0.332520,0.242042


5.016308
28090


In [22]:
# All data is loaded, now compute analytical aspects
r_cut = (2.**(1./6.))

# Get lattice spacing for particle size
def ljForce(r, eps, sigma=1.):
    div = (sigma/r)
    dU = (24. * eps / sigma) * ((2*(div**13)) - (div)**7)
    return dU

# # Lennard-Jones pressure
# def ljPress(r, eps, sigma=1.):
#     phiCP = np.pi / (2. * np.sqrt(3.))
#     div = (sigma/r)
#     dU = (24. * eps / r) * ((2.*(div**12.)) - (div)**6.)
#     # This is just pressure divided by the area of a particle
# #     return (12. * dU / (np.pi * r))
#     return (12. * dU / (np.pi * r * phiCP))

def ljPress(r, pe, eps, sigma=1.):
    phiCP = np.pi / (2. * np.sqrt(3.))
    # This is off by a factor of 1.2...
    ljF = avgCollisionForce(pe)
    return (2. *np.sqrt(3) * ljF / r)
    
def avgCollisionForce(pe, power=1.):
    '''Computed from the integral of possible angles'''
    peCritical = 40.
    if pe < peCritical:
        pe = 0
    else:
        pe -= peCritical
    magnitude = 6.
    # A vector sum of the six nearest neighbors
    magnitude = np.sqrt(28)
#     return (magnitude * (pe**power)) / (np.pi)
#     return (pe * (1. + (8./(np.pi**2.))))
    coeff = 1.92#2.03#3.5#2.03
    #coeff= 0.4053
    return (pe * coeff)

def fStar(pe, epsilon, sigma=1.):
    out = (avgCollisionForce(pe) * sigma) / (24.*epsilon)
    return out
    
def conForRClust(pe, eps):
    out = []
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while ljForce(r, eps) < avgCollisionForce(pe):
            r -= j
        r += j
    out = r
    return out

def nonDimFLJ(r, sigma=1.):
    div = (sigma/r)
    dU = ((2*(div**13)) - (div)**7)
    return dU

def latForFStar(fstar):
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while nonDimFLJ(r) < fstar:
            r -= j
        r += j
    out = r
    return out
    
def latToPhi(latIn):
    '''Read in lattice spacing, output phi'''
    phiCP = np.pi / (2. * np.sqrt(3.))
    return phiCP / (latIn**2)

# From area fraction, get lattice spacing
def phiToLat(phiIn):
    '''Read in phi, output the lattice spacing'''
    phiCP = np.pi / (2. * np.sqrt(3.))
    latCP = 1.
    return np.sqrt(phiCP / phiIn)
    
def compPhiG(pe, a, kap=4.5, sig=1.):
    num = 3. * (np.pi**2) * kap * sig
    den = 4. * pe * a
    return num / den
    
def clustFrac(phi, phiG, a, sig=1.):
    phiL = latToPhi(a)
    ApL = np.pi * (sig**2) / 4.
    Ap = np.pi * (sig**2) / 4.
    num = (phiL*phiG) - (phiL*phi)
    den = (phi*phiG) - (phi*phiL)
    ans = num / den
    return ans

def radCurve(area):
    # From area of circle get curvature
    return np.sqrt(area/np.pi)

def radCirc(circ):
    return circ / (2. * np.pi)

In [23]:
#Constants
kT = 1.0                        # temperature
threeEtaPiSigma = 1.0           # drag coefficient
sigma = 1.0                     # particle diameter
D_t = kT / threeEtaPiSigma      # translational diffusion constant
D_r = (3.0 * D_t) / (sigma**2)  # rotational diffusion constant
tauBrown = (sigma**2) / D_t     # brownian time scale (invariant)



def compPeNet(xf, pes, pef):
    "Given each species activity (pes and pef) and particle fraction (xf), compute net activity (peNet)"
    peNet = (pes * (1.-xf)) + (pef * xf)
    return peNet
def avgCollisionForce(peNet):
    '''Computed from the integral of possible angles'''
    # A vector sum of the six nearest neighbors
    magnitude = np.sqrt(28)
    return (magnitude * peNet) / (np.pi)  
def ljForce(r, eps, sigma=1.):
    '''Compute the Lennard-Jones force'''
    div = (sigma/r)
    dU = (24. * eps / r) * ((2*(div**12)) - (div)**6)
    return dU

# Lennard-Jones pressure
def ljPress(r, pe, eps, sigma=1.):
    '''
    Purpose: Take epsilon (magnitude of lennard-jones force), sigma (particle diameter),
    activity (pe), and separation distance (r) of 2 particles to compute pressure from
    avg compressive active forces from neighbors
    
    Inputs: 
        r: Separation distance in simulation units
        epsilon: magnitude of lennard-jones potential
        pe: activity (peclet number)
        sigma: particle diameter (default=1.0)
    
    Output: Analytical virial pressure (see monodisperse paper for derivation)
    '''
    #Area fraction at HCP
    phiCP = np.pi / (2. * np.sqrt(3.))
    
    # LJ force
    ljF = avgCollisionForce(pe)
    
    return (2. *np.sqrt(3) * ljF / r)

def getLat(peNet, eps):
    '''Get the lattice spacing for any pe'''
    if peNet == 0:
        return 2.**(1./6.)
    out = []
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while ljForce(r, eps) < avgCollisionForce(peNet):
            r -= j
        r += j
    return r  
def latToPhi(latIn):
    '''Read in lattice spacing, output phi'''
    phiCP = np.pi / (2. * np.sqrt(3.))
    return phiCP / (latIn**2)

#Calculate gas phase area fraction
def compPhiG(pe, a, kap=4.5, sig=1.):
    '''
    Purpose: Compute analytical area fraction of the gas phase at steady state
    given activity and lattice spacing
    
    Inputs: 
        pe: net activity (peclet number)
        a: lattice spacing 
        kap: fitting parameter (default=4.5, shown by Redner)
        sig: particle diameter (default=1.0)
    
    Output: Area fraction of the gas phase at steady state
    '''
    num = 3. * (np.pi**2) * kap * sig
    den = 4. * pe * a
    if den>0:
        return num / den
    else:
        return 0
# Calculate dense phase area fraction from lattice spacing
def latToPhi(latIn):
    '''
    Purpose: Compute analytical area fraction of the dense phase given the lattice
    spacing.
    
    Inputs: 
        latIn: lattice spacing
    
    Output: dense phase area fraction
    '''
    phiCP = np.pi / (2. * np.sqrt(3.))
    return phiCP / (latIn**2)
def areaType(Nx, latx):
    Ax = Nx * np.pi * 0.25 * (latx**2)
    return Ax

#Slow activities interested in
pe_a = [0, 100, 150, 200, 250, 350, 450]
#Fast activities interested in
pe_b = [0, 100, 150, 200, 250, 350, 450]

#Particle fraction of slow activities
xA = 50./100.0

#Particle fraction of fast activities
xF = 50./100.0

#Particle fraction of slow activities
xS=1.0-xF

partNum = 50000          # total number of particles
partNumS=xS*partNum      # total number of slow particles
partNumF=xF*partNum      # total number of fast particles

intPhi = 60               # integer system area fraction
phi = float(intPhi)/100.0 # system area fraction
eps = 1.0                 # particle softness

#initialize empty arrays for appending to
press_fast_int = np.array([])
press_slow_int = np.array([])
press_fast_dense = np.array([])
press_slow_dense = np.array([])
pa_pair = np.array([])
pb_pair = np.array([])
pnet_pair = np.array([])
int_width_slow = np.array([])
int_width_fast = np.array([])
int_width_theory = np.array([])
int_width_predict_fastd = np.array([])
int_width_predict_slowd = np.array([])
int_width_fast_int_arr = np.array([])
int_width_slow_int_arr = np.array([])
#Loop over slow activities
for i in range(0, len(pe_a)): 
    
    #Loop over fast activities
    for j in range(0, len(pe_b)):
        
        #Be sure slow activity is less than or equal to fast activity
        if pe_a[i]<=pe_b[j]:
            
            #Compute net activity
            peNet = compPeNet(xF, pe_a[i], pe_b[j])
            
            # Compute lattice spacing based on each activity
            latS = getLat(pe_a[i], eps)
            latF = getLat(pe_b[j], eps)
            latNet = getLat(peNet, eps)
            
            Ns = partNum * (1. - xF)          # number of slow particles
            Nf = partNum - Ns                 # number of fast particles
            
            phiG = compPhiG(peNet, latNet)              # area fraction of gas phase
            NGas = (phiG / phi) * partNum    # number of gas particles
            
            #Compute area fraction of dense phase based on each lattice spacing
            phi_theory = latToPhi(latNet)
            phiS_theory = latToPhi(latS)
            phiF_theory = latToPhi(latF)
            
            #Compute number density of slow and fast particles
            nS_theory = phi_theory /(np.pi/4)
            nF_theory = phi_theory /(np.pi/4)
            
            #Compute number of particles in liquid phase
            Nl = int(round(partNum * ((phi_theory * (phiG - phi)) / (phi * (phiG - phi_theory)))))
            
            #Compute number of particles in gas phase
            NGas = partNum - Nl
            
            # Critical packing fraction in HCP lattice
            phiCP = np.pi / (2. * np.sqrt(3))
            
            # The area is the sum of the particle areas (normalized by close packing density of spheres)
            Al = (Nl * np.pi * (latNet)**2) / (4*phiCP)
            As = (Ns * np.pi * (latNet)**2) / (4*phiCP)
            if As > Al:
                As = Al
            Af = (Nf * np.pi * (latNet)**2) / (4*phiCP)
            if Af > Al:
                Af = Al
            
            # The area for instantiated liquid cluster
            Al_real=Al
            
            # The cluster radius is the square root of liquid area divided by pi
            Rl = np.sqrt(Al_real / np.pi)
            Rs = np.sqrt(As / np.pi)
            Rf = np.sqrt(Af / np.pi)
            
            #The interface width is the difference between the 100% bulk species radius and the total cluster radius
            int_width_fast_int = Rl-Rs
            int_width_slow_int = Rl-Rf
            int_width_fast_int_arr = np.append(int_width_fast_int_arr, int_width_fast_int)
            int_width_slow_int_arr = np.append(int_width_slow_int_arr, int_width_slow_int)
            #Calculate interface aligned active pressure from theory
            press_fast_int = np.append(press_fast_int, int_width_fast_int * nF_theory * 1.0 * pe_b[j])
            press_slow_int = np.append(press_slow_int, int_width_slow_int * nS_theory * 1.0 * pe_a[i])
            
            #Calculate bulk interparticle pressure from theory
            press_slow_dense_val = 4.0 * np.sqrt(3) * peNet / latNet
            press_fast_dense_val = 4.0 * np.sqrt(3) * peNet / latNet
            
            #Append pressures to array
            press_slow_dense = np.append(press_slow_dense, press_slow_dense_val)
            press_fast_dense = np.append(press_fast_dense, press_fast_dense_val)
            int_width_predict_slowd = np.append(int_width_predict_slowd, press_slow_dense_val/(pe_b[j]*nF_theory))
            int_width_predict_fastd = np.append(int_width_predict_fastd, press_fast_dense_val/(pe_a[i]*nS_theory))
            
            #Append activities to array
            pa_pair = np.append(pa_pair, pe_a[i])
            pb_pair = np.append(pb_pair, pe_b[j])
            pnet_pair = np.append(pnet_pair, pe_b[j]*0.5 + pe_a[i]*0.5)
            
            #Append interface widths to array
            int_width_fast = np.append(int_width_fast, int_width_fast_int)
            int_width_slow = np.append(int_width_slow, int_width_slow_int)
            
            #Calculate dense phase pressure again and constants for predicting what interface width is for balancing pressure
            curPLJ = ljPress(latNet, peNet, eps)

            alpha_max = 0.5
            I_arr = 3.0
            int_width = (np.sqrt(3)/(2*alpha_max)) * (curPLJ/peNet) * (latNet **2) * I_arr
            int_width_theory = np.append(int_width_theory, int_width)

<ipython-input-23-0d8571e0afad>:214: RuntimeWarning: invalid value encountered in double_scalars
  int_width_predict_slowd = np.append(int_width_predict_slowd, press_slow_dense_val/(pe_b[j]*nF_theory))
<ipython-input-23-0d8571e0afad>:215: RuntimeWarning: invalid value encountered in double_scalars
  int_width_predict_fastd = np.append(int_width_predict_fastd, press_fast_dense_val/(pe_a[i]*nS_theory))
<ipython-input-23-0d8571e0afad>:231: RuntimeWarning: invalid value encountered in double_scalars
  int_width = (np.sqrt(3)/(2*alpha_max)) * (curPLJ/peNet) * (latNet **2) * I_arr
<ipython-input-23-0d8571e0afad>:194: RuntimeWarning: invalid value encountered in sqrt
  Rl = np.sqrt(Al_real / np.pi)
<ipython-input-23-0d8571e0afad>:195: RuntimeWarning: invalid value encountered in sqrt
  Rs = np.sqrt(As / np.pi)
<ipython-input-23-0d8571e0afad>:196: RuntimeWarning: invalid value encountered in sqrt
  Rf = np.sqrt(Af / np.pi)
<ipython-input-23-0d8571e0afad>:215: RuntimeWarning: divide by zero enc

In [24]:
# Let's use analytical theory and kinetic theory to get cluster radius
epsRange = [1., 0.1, 0.01, 0.001, 0.0001]
# epsRange = [0.0001, 0.001, 0.01, 0.1, 1.]
peRange = np.arange(0., 700., 1.)
phiRange = [0.45, 0.55, 0.65]
N = 100000.
norm = 10.**0.
# norm = 1.

phiCP = np.pi / (2. * np.sqrt(3))
lat = []
pColl = []
pLJ = []
cfs = []
Rls = []
peCrit = []
phiGs = []
phiCPs = []
a_box = []
l_box = []
for b in range(0, len(phiRange)):
    lat.append([])
    phiCPs.append([])
    pColl.append([])
    pLJ.append([])
    cfs.append([])
    Rls.append([])
    phiGs.append([])
    peCrit.append([])
    a_box.append(N * np.pi * 0.25 / phiRange[b])
    l_box.append(np.sqrt(a_box[-1]))
    for i in range(0, len(epsRange)):
        lat[b].append([])
        phiCPs[b].append([])
        pColl[b].append([])
        pLJ[b].append([])
        cfs[b].append([])
        Rls[b].append([])
        phiGs[b].append([])
        for j in range(0, len(peRange)):
            # Compute lattice spacing
        
            lat[b][i].append(conForRClust(peRange[j], epsRange[i]))
            phiCPs[b][i].append(latToPhi(lat[b][i][-1]))
            # Compute pressure
            #curPLJ = ljPress(lat[b][i][-1], epsRange[i]) / (np.pi * (lat[b][i][-1]**2) * 0.25 * phiCP)
            #curPLJ = ljPress(lat[b][i][-1], epsRange[i]) / (np.pi * (lat[b][i][-1]**2) * 0.25 * phiCP * (lat[b][i][-1]**(0.5)) * 1.25)
            curPLJ = ljPress(lat[b][i][-1], peRange[j], epsRange[i])
            
            # Append to list
            pLJ[b][i].append(curPLJ/(norm))

            # Compute cluster fraction
            phiG = compPhiG(peRange[j], lat[b][i][-1])
            phiGs[b][i].append(phiG)
            if peRange[j] > 35.:
                cf = clustFrac(phiRange[b], phiG, lat[b][i][-1])
                if cf < 0. or cf > 1.:
                    cf = 0.
            else:
                cf = 0
            cfs[b][i].append(cf)

            # Get the critical activity
            if j > 0:
                if cfs[b][i][-2] == 0. and cfs[b][i][-1] > 0.:
                    peCrit[b].append(peRange[j])

            # Get the radius (for some N)
            Nl = cfs[b][i][-1] * N
            Al = Nl * ((np.pi * (lat[b][i][-1]**2))/(4*phiCP))
            Rl = np.sqrt(Al / (np.pi))
            Rls[b][i].append(Rl)

In [25]:
align_peA=np.array([])
align_peB=np.array([])
align_peNet=np.array([])
first_align_peNet=np.array([])
first_interpart_peNet=np.array([])
first_interpart_peA=np.array([])
first_interpart_peB=np.array([])

align_xA=np.array([])
align_eps=np.array([])
align_pnum=np.array([])
align_phi=np.array([])
align_press_arr = np.array([])
first_align_press = np.array([])
first_bulk_press = np.array([])
first_align_peA=np.array([])
first_align_peB=np.array([])

interpart_peA=np.array([])
interpart_peB=np.array([])
interpart_peNet=np.array([])
interpart_xA=np.array([])
interpart_eps=np.array([])
interpart_pnum=np.array([])
interpart_phi=np.array([])
interpart_press=np.array([])
interpart_press_expand=np.array([])
avg_shear=np.array([])
fastCol = '#e31a1c'
slowCol = '#081d58'

avg_rad = np.array([])
avg_width = np.array([])
avg_clust = np.array([])
avg_circularity = np.array([])

bulk_lat_time_mean = np.array([])
int_lat_time_mean = np.array([])
all_lat_time_mean = np.array([])
bulk_lat_time_std = np.array([])
int_lat_time_std = np.array([])
all_lat_time_std = np.array([])


avg_bulk_nA = np.array([])
avg_bulk_nB = np.array([])
avg_bulk_n = np.array([])
avg_dense_nA = np.array([])
avg_dense_nB = np.array([])
avg_dense_n = np.array([])
avg_int_n = np.array([])
avg_int_nA = np.array([])
avg_int_nB = np.array([])
avg_gas_nA = np.array([])
avg_gas_nB = np.array([])
avg_gas_n = np.array([])

avg_int_theory_arr = np.array([])
avg_bulk_theory_arr = np.array([])
def ljForce(r, eps, sigma=1.):
    div = (sigma/r)
    dU = (24. * eps / sigma) * ((2*(div**13)) - (div)**7)
    return dU

def avgCollisionForce(pe, power=1.):
    '''Computed from the integral of possible angles'''
    peCritical = 40.
    if pe < peCritical:
        pe = 0
    else:
        pe -= peCritical
    magnitude = 6.
    # A vector sum of the six nearest neighbors
    magnitude = np.sqrt(28)
#     return (magnitude * (pe**power)) / (np.pi)
#     return (pe * (1. + (8./(np.pi**2.))))
    coeff = 1.874#3.0#1.92#2.03#3.5#2.03
    #coeff= 0.4053
    return (pe * coeff)

def conForRClust(pe, eps):
    out = []
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while ljForce(r, eps) < avgCollisionForce(pe):
            r -= j
        r += j
    out = r
    return out
def densProbability(r, activity_net, activity_slow):
    "Using similar triangles to find sides"
    gas_dense_dif_phi = 0.866 * np.log10(activity_net - 46.993) - 0.443
    rate_decay = -6.151 * np.log10(activity_net-49.921) - 4.392
    mid_point = 0.044 * np.log10(activity_slow-49.893) + 0.836
    gas_phi = -0.26 * np.log10(activity_slow-41.742)+0.783
    
    num_dens_r = ((gas_dense_dif_phi / (1+np.exp(-rate_decay * (r-mid_point)))) + gas_phi)
    
    return num_dens_r

def alignProbability(r, activity_net, activity_slow):
    "Using similar triangles to find sides"
    max_align = 0.2019 * np.log10(activity_net - 41.2803) - 0.1885
    mid_point = 0.0492 * np.log10(activity_slow - 47.0061) + 0.8220
    std_dev = 0.1057
    align_r = max_align * np.exp(-(r-mid_point)**2/(2*std_dev**2))
    
    return align_r

for i in range(0, len(all_dens)):
    if params['peB'][i]>=50:
        if all_dens[i].empty:
            continue
        for k in range(0, len(all_pres)):
            if params2['peA'][k]==params['peA'][i]:
                if params2['peB'][k]==params['peB'][i]:
                    if all_pres[k].empty:
                        continue
                    for q in range(0, len(all_pres_new2)):
                        if all_pres_new2[q].empty:
                            continue
                        if params4['peA'][q]==params['peA'][i]:
                            if params4['peB'][q]==params['peB'][i]:
                                for s in range(0, len(all_pres_new3)):
                                    if all_pres_new3[s].empty:
                                        continue
                                    if params5['peA'][s]==params['peA'][i]:
                                        if params5['peB'][s]==params['peB'][i]:
                                            for t in range(0, len(all_pres_new)):
                                                if all_pres_new[t].empty:
                                                    continue
                                                if params3['peA'][t]==params['peA'][i]:
                                                    if params3['peB'][t]==params['peB'][i]:
                                                        for u in range(0, len(all_pres_new4)):
                                                            if all_pres_new4[u].empty:
                                                                continue
                                                            if params6['peA'][u]==params['peA'][i]:
                                                                if params6['peB'][u]==params['peB'][i]:
                                                                    print(params['peA'][i])
                                                                    print(params['peB'][i])
                                                                    time_i = all_dens[i]['tauB']
                                                                    time_k = all_pres[k]['Timestep']
                                                                    time_q = all_pres_new2[q]['tauB']
                                                                    time_s = all_pres_new3[s]['tauB']
                                                                    time_t = all_pres_new[t]['tauB']
                                                                    time_u = all_pres_new4[u]['tauB']

                                                                    #Instantiate arrays for reading in interface pressure .txt file
                                                                    final_time_i =np.array([])
                                                                    final_align_press = np.array([])
                                                                    align_press_total = 0
                                                                    align_press_vals=0

                                                                    #Instantiate arrays for reading in bulk pressure .txt file
                                                                    bulk_time =np.array([])
                                                                    bulk_press_time = np.array([])
                                                                    bulk_press_total = 0
                                                                    bulk_press_total_expand = 0
                                                                    bulk_press_vals=0
                                                                    shear_press_expand=0

                                                                    #Instantiate arrays for reading in number density .txt file
                                                                    bulk_nA_arr = np.array([])
                                                                    bulk_nB_arr = np.array([])
                                                                    bulk_n_arr = np.array([])
                                                                    int_n_arr = np.array([])
                                                                    int_nA_arr = np.array([])
                                                                    int_nB_arr = np.array([])
                                                                    gas_nA_arr = np.array([])
                                                                    gas_n_arr = np.array([])
                                                                    gas_nB_arr = np.array([])
                                                                    dense_nA_arr = np.array([])
                                                                    dense_nB_arr = np.array([])
                                                                    dense_n_arr = np.array([])

                                                                    #Instantiate arrays for reading in lattice spacing .txt file
                                                                    bulk_lat_mean = np.array([])
                                                                    int_lat_mean = np.array([])
                                                                    all_lat_mean = np.array([])
                                                                    bulk_lat_std = np.array([])
                                                                    int_lat_std = np.array([])
                                                                    all_lat_std = np.array([])

                                                                    #Instantiate arrays for reading in radial measurement .txt file
                                                                    num_dens_arr_temp = np.zeros(50)
                                                                    align_arr_temp = np.zeros(50)
                                                                    press_arr_temp = np.zeros(50)
                                                                    xA_arr_temp = np.zeros(50)
                                                                    xB_arr_temp = np.zeros(50)
                                                                    temp_count = np.zeros(50)
                                                                    temp_count_count = np.zeros(50)

                                                                    radius_arr_final = np.linspace(0,1.6,num=50)
                                                                    num_dens_arr_final = np.zeros(50)
                                                                    align_arr_final = np.zeros(50)
                                                                    xA_arr_final = np.zeros(50)
                                                                    xB_arr_final = np.zeros(50)
                                                                    press_arr_final = np.zeros(50)
                                                                    final_count = np.zeros(50)
                                                                    final_count_count = np.zeros(50)

                                                                    press_theory_time = np.array([])
                                                                    press_theory_val = np.array([])
                                                                    time_arr = np.array([])

                                                                    bub_width_arr = np.array([])
                                                                    bub_area_arr = np.array([])
                                                                    bub_surface_area_arr = np.array([])
                                                                    int_width_arr = np.array([])
                                                                    clust_arr = np.array([])
                                                                    
                                                                    time_dens = np.array([])
                                                                    time_width = np.array([])
                                                                    time_lat = np.array([])
                                                                    #Loop over time in interface pressure .txt file
                                                                    for j in range(0, len(time_i)):

                                                                        #Current time in loop
                                                                        time = time_i[j]

                                                                        #Read values and set variables from interface pressure .txt file
                                                                        align_press = all_dens[i]['press_align'][j]

                                                                        if align_press > 0:
                                                                            final_time_i = np.append(final_time_i, all_dens[i]['tauB'].iloc[j])
                                                                            final_align_press = np.append(final_align_press, align_press)

                                                                            align_press_total += align_press
                                                                            align_press_vals += 1

                                                                        #Read values and set variables from bulk pressure.txt files

                                                                        time_id_k = np.where(time_k==time)[0]
                                                                        if len(time_id_k)>0:
                                                                            time_id_k=time_id_k[0]

                                                                            bulk_trace = (all_pres[k]['bulkSigXX'].iloc[time_id_k]+all_pres[k]['bulkSigYY'].iloc[time_id_k])/2


                                                                            bulk_press = bulk_trace / (all_pres[k]['bulkArea'].iloc[time_id_k])

                                                                            bulk_trace_expand = (all_pres[k]['bulkSigXX'].iloc[time_id_k]+all_pres[k]['bulkSigXY'].iloc[time_id_k]+all_pres[k]['bulkSigYX'].iloc[time_id_k]+all_pres[k]['bulkSigYY'].iloc[time_id_k])/2
                                                                            bulk_press_expand = bulk_trace_expand / (all_pres[k]['bulkArea'].iloc[time_id_k])

                                                                            shear_stress = (all_pres[k]['bulkSigXY'].iloc[time_id_k]+all_pres[k]['bulkSigYX'].iloc[time_id_k])/2
                                                                            shear_press = shear_stress / (all_pres[k]['bulkArea'].iloc[time_id_k])
                                                                            if bulk_press>0:
                                                                                bulk_time = np.append(bulk_time, all_pres[k]['Timestep'].iloc[time_id_k])
                                                                                bulk_press_time = np.append(bulk_press_time, bulk_press/2)
                                                                                shear_press_expand +=shear_press
                                                                                bulk_press_total += bulk_press
                                                                                bulk_press_total_expand += bulk_press_expand
                                                                                bulk_press_vals += 1

                                                                        #Read values and set variables from number density .txt files
                                                                        time_id_q = np.where(time_q==time)[0]
                                                                        if len(time_id_q)>0:
                                                                            time_dens = np.append(time_dens, time)
                                                                            bin_size = all_pres_new2[q]['sizeBin'][time_id_q]
                                                                            bin_area = bin_size**2
                                                                            bulk_nA_arr = np.append(bulk_nA_arr, all_pres_new2[q]['Na_bulk'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_bulk'][time_id_q]))
                                                                            bulk_nB_arr = np.append(bulk_nB_arr, all_pres_new2[q]['Nb_bulk'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_bulk'][time_id_q]))
                                                                            bulk_n_arr = np.append(bulk_n_arr, (all_pres_new2[q]['Nb_bulk'][time_id_q]+all_pres_new2[q]['Na_bulk'][time_id_q]) / (bin_area * all_pres_new2[q]['NBin_bulk'][time_id_q]))
                                                                            int_n_arr = np.append(int_n_arr, (all_pres_new2[q]['Na_int'][time_id_q]+all_pres_new2[q]['Nb_int'][time_id_q]) / (bin_area * all_pres_new2[q]['NBin_int'][time_id_q]))
                                                                            int_nA_arr = np.append(int_nA_arr, all_pres_new2[q]['Na_int'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_int'][time_id_q]))
                                                                            int_nB_arr = np.append(int_nB_arr, all_pres_new2[q]['Nb_int'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_int'][time_id_q]))
                                                                            gas_n_arr = np.append(gas_n_arr, (all_pres_new2[q]['Na_gas'][time_id_q] + all_pres_new2[q]['Nb_gas'][time_id_q]) / (bin_area * all_pres_new2[q]['NBin_gas'][time_id_q]))
                                                                            gas_nA_arr = np.append(gas_nA_arr, all_pres_new2[q]['Na_gas'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_gas'][time_id_q]))
                                                                            gas_nB_arr = np.append(gas_nB_arr, all_pres_new2[q]['Nb_gas'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_gas'][time_id_q]))
                                                                            dense_n_arr = np.append(dense_n_arr, (all_pres_new2[q]['Na_int'][time_id_q]+all_pres_new2[q]['Nb_int'][time_id_q]+all_pres_new2[q]['Na_bulk'][time_id_q]+all_pres_new2[q]['Nb_bulk'][time_id_q]) / (bin_area * (all_pres_new2[q]['NBin_int'][time_id_q]+all_pres_new2[q]['NBin_bulk'][time_id_q])))
                                                                            dense_nA_arr = np.append(dense_nA_arr, (all_pres_new2[q]['Na_int'][time_id_q]+all_pres_new2[q]['Na_bulk'][time_id_q]) / (bin_area * (all_pres_new2[q]['NBin_int'][time_id_q]+all_pres_new2[q]['NBin_bulk'][time_id_q])))
                                                                            dense_nB_arr = np.append(dense_nB_arr, (all_pres_new2[q]['Nb_int'][time_id_q]+all_pres_new2[q]['Nb_bulk'][time_id_q]) / (bin_area * (all_pres_new2[q]['NBin_int'][time_id_q]+all_pres_new2[q]['NBin_bulk'][time_id_q])))

                                                                        #Read values and set variables from lattice spacing .txt files
                                                                        time_id_u = np.where(time_u==time)[0]
                                                                        if len(time_id_u)>0:
                                                                            time_id_u = time_id_u[0]
                                                                            time_lat = np.append(time_lat, time)
                                                                            bulk_lat_mean = np.append(bulk_lat_mean, all_pres_new4[u]['lat_mean_bulk'][time_id_u])
                                                                            bulk_lat_std = np.append(bulk_lat_std, all_pres_new4[u]['lat_std_bulk'][time_id_u])
                                                                            int_lat_mean = np.append(int_lat_mean, all_pres_new4[u]['lat_mean_int'][time_id_u])
                                                                            int_lat_std = np.append(int_lat_std, all_pres_new4[u]['lat_std_int'][time_id_u])
                                                                            all_lat_mean = np.append(all_lat_mean, all_pres_new4[u]['lat_mean_all'][time_id_u])
                                                                            all_lat_std = np.append(all_lat_std, all_pres_new4[u]['lat_std_all'][time_id_u])


                                                                        #Read values and set variables from radial measurement .txt files
                                                                        time_id_s = np.where(time_s==time)[0]
                                                                        time_id_t = np.where(time_t==time)[0]

                                                                        max_size = np.amax(all_pres_new[t]['clust_size'])
                                                                        for z in range(0, len(time_id_t)):
                                                                            #if all_pres_new[t]['clust_size'][time_id_t[z]]>=0.95*max_size:
                                                                            if all_pres_new[t]['int_id'][time_id_t[z]] == all_pres_new[t]['bub_id'][time_id_t[z]]:
                                                                                time_width = np.append(time_width, time)
                                                                                bub_width_arr = np.append(bub_width_arr, all_pres_new[t]['radius'][time_id_t[z]])
                                                                                bub_surface_area_arr = np.append(bub_surface_area_arr, all_pres_new[t]['sa_ext'][time_id_t[z]])
                                                                                int_width_arr = np.append(int_width_arr, all_pres_new[t]['edge_width'][time_id_t[z]])
                                                                                clust_arr = np.append(clust_arr, all_pres_new[t]['clust_size'][time_id_t[z]])
                                                                        avg_rad_temp = np.mean(bub_width_arr)
                                                                        uncertain = np.where(((0.9*avg_rad_temp)<=bub_width_arr) & (bub_width_arr<=(1.1*avg_rad_temp)))[0]
                                                                        bub_width_arr = bub_width_arr[uncertain]
                                                                        time_width = time_width[uncertain]
                                                                        bub_surface_area_arr = bub_surface_area_arr[uncertain]
                                                                        int_width_arr = int_width_arr[uncertain]
                                                                        clust_arr = clust_arr[uncertain]
                                                                        for y in time_id_s:
                                                                            if (len(time_id_s)>0) & (len(time_id_t)>0):

                                                                                if len(time_id_t)==1:
                                                                                    edge_width = all_pres_new[t]['radius'][time_id_t[0]]
                                                                                    nB = all_pres_new[t]['Nb'][time_id_t[0]]
                                                                                    nA = all_pres_new[t]['Na'][time_id_t[0]]
                                                                                elif len(time_id_t)>1:
                                                                                    edge_width = np.max(all_pres_new[t]['radius'][time_id_t])
                                                                                    nB = np.max(all_pres_new[t]['Nb'][time_id_t])
                                                                                    nA = np.max(all_pres_new[t]['Na'][time_id_t])
                                                                                if time not in time_arr:
                                                                                    time_arr = np.append(time_arr, time)  
                                                                                    if len(time_arr)>=2:

                                                                                        for x in range(0, len(radius_arr_final)-1):
                                                                                            rad_where = np.where((radius_arr_final[x]<=radius_arr) & (radius_arr<radius_arr_final[x+1]))[0]
                                                                                            if len(rad_where)>0:
                                                                                                nonzero_align = np.where(align_arr[rad_where]!=0)[0]
                                                                                                if len(nonzero_align)>0:
                                                                                                    press_arr_final[x]+=np.mean(press_arr[rad_where][nonzero_align])
                                                                                                    align_arr_final[x]+=np.mean(align_arr[rad_where][nonzero_align])
                                                                                                    num_dens_arr_final[x]+=np.mean(num_dens_arr[rad_where][nonzero_align])
                                                                                                    xA_arr_final[x]+=np.mean(xA_arr[rad_where][nonzero_align])
                                                                                                    xB_arr_final[x]+=np.mean(xB_arr[rad_where][nonzero_align])
                                                                                                    final_count[x]+=1
                                                                                                    final_count_count[x]+=len(rad_where)

                                                                                                    press_arr_temp[x]=np.mean(press_arr[rad_where][nonzero_align])
                                                                                                    align_arr_temp[x]=np.mean(align_arr[rad_where][nonzero_align])
                                                                                                    xA_arr_temp[x]=np.mean(xA_arr[rad_where][nonzero_align])
                                                                                                    xB_arr_temp[x]=np.mean(xB_arr[rad_where][nonzero_align])
                                                                                                    num_dens_arr_temp[x]=np.mean(num_dens_arr[rad_where][nonzero_align])

                                                                                        peNet_int = (int_nA_arr[-1]/int_n_arr[-1]) * float(params['peA'][i]) + (int_nB_arr[-1]/int_n_arr[-1]) * float(params['peB'][i])
                                                                                        peNet = 0.5 * float(params['peA'][i]) + 0.5 * float(params['peB'][i])

                                                                                        dens = densProbability(radius_arr_final, peNet, params['peB'][i])
                                                                                        align = alignProbability(radius_arr_final, peNet, params['peB'][i])

                                                                                        press = dens * align * ((xA_arr_temp * params['peA'][i] + xB_arr_temp * params['peB'][i]))
                                                                                        press_int = 0
                                                                                        for m in range(1, len(press)):
                                                                                            press_int += ((press[m-1]+press[m])/2)*(radius_arr_final[m]-radius_arr_final[m-1]) * edge_width
                                                                                        press_theory_val = np.append(press_theory_val, press_int)
                                                                                        press_theory_time = np.append(press_theory_time, time)
                                                                                        


                                                                                    radius_arr = np.array([])
                                                                                    align_arr = np.array([])
                                                                                    num_dens_arr = np.array([])
                                                                                    press_arr = np.array([])
                                                                                    xA_arr = np.array([])
                                                                                    xB_arr = np.array([])
                                                                                    radius_arr = np.append(radius_arr, all_pres_new3[s]['r_max'][y]/edge_width)   
                                                                                    align_arr = np.append(align_arr, all_pres_new3[s]['align'][y])   
                                                                                    xA_arr = np.append(xA_arr, all_pres_new3[s]['num_densA'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                                    xB_arr = np.append(xB_arr, all_pres_new3[s]['num_densB'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                                    num_dens_arr = np.append(num_dens_arr, all_pres_new3[s]['num_dens'][y])   
                                                                                    press_arr = np.append(press_arr, all_pres_new3[s]['press'][y])   
                                                                                else:
                                                                                    radius_arr = np.append(radius_arr, all_pres_new3[s]['r_max'][y]/edge_width)   
                                                                                    area_slice = np.pi*(all_pres_new3[s]['r_max'][j]**2 - all_pres_new3[s]['r_min'][y]**2)
                                                                                    align_arr = np.append(align_arr, all_pres_new3[s]['align'][y])   
                                                                                    num_dens_arr = np.append(num_dens_arr, all_pres_new3[s]['num_dens'][y])   
                                                                                    xA_arr = np.append(xA_arr, all_pres_new3[s]['num_densA'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                                    xB_arr = np.append(xB_arr, all_pres_new3[s]['num_densB'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                                    press_arr = np.append(press_arr, all_pres_new3[s]['press'][y]) 



                                                                    #Time-average values for sufficiently long interface pressure .txt files
                                                                    if align_press_vals > 50:        
                                                                        avg_press = (align_press_total/align_press_vals)
                                                                        align_press_arr = np.append(align_press_arr, avg_press)
                                                                        align_peA=np.append(align_peA, params['peA'][i])
                                                                        align_peB=np.append(align_peB, params['peB'][i])
                                                                        align_peNet=np.append(align_peNet, params['peB'][i] * (1-params['xA'][i]/100) + params['peA'][i] * (params['xA'][i]/100))
                                                                        align_xA=np.append(align_xA, params['xA'][i])
                                                                        align_phi=np.append(align_phi, params['phi'][i])
                                                                        align_eps=np.append(align_eps, params['eps'][i])

                                                                    #Time-average values for sufficiently long bulk pressure .txt files
                                                                    if bulk_press_vals > 50:        
                                                                        avg_shear = np.append(avg_shear, shear_press_expand/bulk_press_vals)
                                                                        avg_press = (bulk_press_total/(2*bulk_press_vals))
                                                                        avg_press_expand = (bulk_press_total_expand/bulk_press_vals)
                                                                        interpart_press=np.append(interpart_press, avg_press)
                                                                        interpart_press_expand=np.append(interpart_press_expand, avg_press_expand)
                                                                        interpart_peA=np.append(interpart_peA, params2['peA'][k])
                                                                        interpart_peB=np.append(interpart_peB, params2['peB'][k])
                                                                        interpart_peNet=np.append(interpart_peNet, params2['peB'][k] * (1-params2['xA'][k]/100) + params2['peA'][k] * (params2['xA'][k]/100))
                                                                        interpart_xA=np.append(interpart_xA, params2['xA'][k])
                                                                        interpart_phi=np.append(interpart_phi, params2['phi'][k])
                                                                        interpart_eps=np.append(interpart_eps, params2['eps'][k])
                                                                    #Time-average values for sufficiently long bubble composition .txt files
                                                                    if len(bub_width_arr)>0:
                                                                        nonzero = np.where(bub_surface_area_arr>0)[0]
                                                                        avg_rad = np.append(avg_rad, np.mean(bub_width_arr[nonzero]))
                                                                        avg_circularity = np.append(avg_circularity, np.mean((np.pi*(bub_width_arr[nonzero]**2)*4*np.pi)/bub_surface_area_arr[nonzero]**2))
                                                                        avg_width = np.append(avg_width, np.mean(int_width_arr[nonzero]))
                                                                        avg_clust = np.append(avg_clust, np.mean(clust_arr[nonzero]))

                                                                    bulk_lat_time_mean = np.append(bulk_lat_time_mean, np.mean(bulk_lat_mean))
                                                                    int_lat_time_mean = np.append(int_lat_time_mean, np.mean(int_lat_mean))
                                                                    all_lat_time_mean = np.append(all_lat_time_mean, np.mean(all_lat_mean))

                                                                    bulk_lat_time_std = np.append(bulk_lat_time_std, np.mean(bulk_lat_std))
                                                                    int_lat_time_std = np.append(int_lat_time_std, np.mean(int_lat_std))
                                                                    all_lat_time_std = np.append(all_lat_time_std, np.mean(all_lat_std))
                                                                    
                                                                    #Time-average values for sufficiently long phase composition .txt files
                                                                    avg_bulk_nA = np.append(avg_bulk_nA, np.mean(bulk_nA_arr))
                                                                    avg_bulk_n = np.append(avg_bulk_n, np.mean(bulk_n_arr))
                                                                    avg_bulk_nB = np.append(avg_bulk_nB, np.mean(bulk_nB_arr))
                                                                    
                                                                    
                                                                    avg_int_n = np.append(avg_int_n, np.mean(int_n_arr))
                                                                    avg_int_nA = np.append(avg_int_nA, np.mean(int_nA_arr))
                                                                    avg_int_nB = np.append(avg_int_nB, np.mean(int_nB_arr))
                                                                    
                                                                    avg_gas_n = np.append(avg_gas_n, np.mean(gas_n_arr))
                                                                    avg_gas_nA = np.append(avg_gas_nA, np.mean(gas_nA_arr))
                                                                    avg_gas_nB = np.append(avg_gas_nB, np.mean(gas_nB_arr))
                                                                    
                                                                    avg_dense_nA = np.append(avg_dense_nA, np.mean(dense_nA_arr))
                                                                    avg_dense_n = np.append(avg_dense_n, np.mean(dense_n_arr))
                                                                    avg_dense_nB = np.append(avg_dense_nB, np.mean(dense_nB_arr))
                                                                            
                                                                    for m in range(0, len(align_arr_temp)):
                                                                        if temp_count[m]>0:
                                                                            align_arr_final[m]=align_arr_final[m]/final_count[m]
                                                                            num_dens_arr_final[m]=num_dens_arr_final[m]/final_count[m]
                                                                            xA_arr_final[m]=xA_arr_final[m]/final_count[m]
                                                                            xB_arr_final[m]=xB_arr_final[m]/final_count[m]

                                                                    peNet_int = (int_nA_arr/int_n_arr) * float(params['peA'][i]) + (int_nB_arr/int_n_arr) * float(params['peB'][i])
                                                                    peNet_dense = (dense_nA_arr/dense_n_arr) * float(params['peA'][i]) + (dense_nB_arr/dense_n_arr) * float(params['peB'][i])
                                                                    peNet_bulk = (bulk_nA_arr/bulk_n_arr) * float(params['peA'][i]) + (bulk_nB_arr/bulk_n_arr) * float(params['peB'][i])

                                                                    latNet_int = np.zeros(len(peNet_int))

                                                                    #dense_theory = np.zeros(len(peNet_int))
                                                                    phi_cp = np.pi / (2*np.sqrt(3))
                                                                    for z in range(0, len(peNet_int)):
                                                                        latNet_int[z] = conForRClust(peNet_int[z]-50, 1.0)
                                                                    dense_theory = phi_cp / latNet_int**2
                                                                    gas_theory = (3 * np.pi**2 * 4.5/4) * ((latNet_int * peNet_int) ** (-1))
                                                                    clust_val = 50000* (((gas_theory-0.6)*dense_theory)/(0.6*(gas_theory-dense_theory)))
                                                                    rad_val =  ((clust_val / (4*phi_cp))**0.5) * latNet_int
                                                                    theoryss = (peNet_int-50) * 2 * 1.874 * np.sqrt(3) / latNet_int

                                                                    fsize=10
                                                                    mpl.rcParams.update({'font.size': 13})
                                                                    mkSz = [0.1, 0.1, 0.15, 0.1, 0.1]
                                                                    msz=40
                                                                    yellow = ("#fec44f")
                                                                    green = ("#77dd77")
                                                                    red = ("#ff6961")
                                        
                                                                    dens_sim2 = clust_arr * (np.pi/4) / (np.pi * bub_width_arr**2)


                                                                    nonzero_bulk = np.where(bulk_press_time>0)[0]
                                                                    nonzero_align = np.where(final_align_press>0)[0]

                                                                    if len(nonzero_bulk)>10:
                                                                        if len(nonzero_align)>10:
                                                                            slowSim = '#de2d26'
                                                                            slowTheory = '#fc9272'
                                                                            fastSim = '#3182bd'
                                                                            fastTheory = '#9ecae1'
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))
                                                                            interpart_max = np.max(peNet_int)
                                                                            align_max = np.max(peNet_bulk)
                                                                            interpart_min = np.min(peNet_int)
                                                                            align_min = np.min(peNet_bulk)
                                                                            
                                                                            fastTheory = '#081d58'
                                                                            slowTheory = '#e31a1c'
                                                                            
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 25
                                                                            else:
                                                                                plot_max = interpart_max + 25
                                                                            
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 25
                                                                            else:
                                                                                plot_min = interpart_min - 25
                                                                            step = np.floor((plot_max - plot_min)/6)


                                                                            plt.plot(time_dens, peNet_int,
                                                                                           c=slowTheory, lw=1.8, ls='dotted', label='Interface')
                                                                            plt.plot(time_dens, peNet_bulk,
                                                                                           c=fastTheory, lw=1.8, ls='dotted', label='Bulk')
                                                                            plt.plot(time_dens, peNet_dense,
                                                                                           c=slowSim, lw=1.8, ls='dotted', label='Dense')
                                                                            ax1.set_xlim(0, 200)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Net Activity ($\mathrm{Pe}_\mathrm{Net}$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=20)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=10)
                                                                            ax1.xaxis.set_minor_locator(loc)


                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=step)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=int(step/2))
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            # Left middle plot
                                                                            plt.legend(loc='upper right', fontsize=fsize*2.0)

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.savefig('/Volumes/External/pe_time2/slow_out_pa' + str(int(params['peA'][i])) + '_pb'+str(int(params['peB'][i])), dpi=100)
                                                                            plt.close()
                                                                            
                                                                            
                                                                            
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))
                                                                            interpart_max = np.max(latNet_int)
                                                                            align_max = np.max(all_lat_mean)
                                                                            interpart_min = np.min(latNet_int)
                                                                            align_min = np.min(all_lat_mean)
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 0.1
                                                                            else:
                                                                                plot_max = interpart_max + 0.1
                                                                                
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 0.1
                                                                            else:
                                                                                plot_min = interpart_min - 0.1

                                                                            
                                                                            step = np.round((plot_max - plot_min)/6, 2)



                                                                            plt.plot(time_dens, latNet_int,
                                                                                           c=slowTheory, lw=1.8, ls='dotted', label='Theory')
                                                                            plt.plot(time_lat, all_lat_mean,
                                                                                           c=fastTheory, lw=1.8, ls='--', label='Simulation')
                                                                            ax1.set_xlim(0, 200)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Lattice Spacing ($a$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=20)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=10)
                                                                            ax1.xaxis.set_minor_locator(loc)


                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=step)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=step/2)
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            # Left middle plot
                                                                            plt.legend(loc='upper right', fontsize=fsize*2.0)

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.savefig('/Volumes/External/lat_time2/slow_out_pa' + str(int(params['peA'][i])) + '_pb'+str(int(params['peB'][i])), dpi=100)
                                                                            plt.close()
                                                                            
                                                                            
                                                                            
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))

                                                                            interpart_max = np.max(bub_width_arr)
                                                                            align_max = np.max(rad_val)
                                                                            interpart_min = np.min(bub_width_arr)
                                                                            align_min = np.min(rad_val)
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 10
                                                                            else:
                                                                                plot_max = interpart_max + 10
                                                                                
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 10
                                                                            else:
                                                                                plot_min = interpart_min - 10
                                                                            step = np.floor((plot_max - plot_min)/6)


                                                                            plt.plot(time_dens, rad_val,
                                                                                           c=slowTheory, lw=1.8, ls='dotted', label='Theory')
                                                                            plt.plot(time_width, bub_width_arr,
                                                                                           c=fastTheory, lw=1.8, ls='--', label='Simulation')
                                                                            ax1.set_xlim(0, 200)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Cluster Radius ($r_\mathrm{c}$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=20)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=10)
                                                                            ax1.xaxis.set_minor_locator(loc)


                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=step)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=int(step/2))
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            # Left middle plot
                                                                            plt.legend(loc='upper right', fontsize=fsize*2.0)

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.savefig('/Volumes/External/rad_time2/slow_out_pa' + str(int(params['peA'][i])) + '_pb'+str(int(params['peB'][i])), dpi=100)
                                                                            plt.close()
                                                                            
                                                                            
                                                                            
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))
                                                                            interpart_max = np.max(dens_sim2)
                                                                            align_max = np.max(dense_theory)
                                                                            interpart_min = np.min(dens_sim2)
                                                                            align_min = np.min(dense_theory)
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 0.1
                                                                            else:
                                                                                plot_max = interpart_max + 0.1
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 0.1
                                                                            else:
                                                                                plot_min = interpart_min - 0.1
                                                                            step = np.round((plot_max - plot_min)/6, 2)


                                                                            plt.plot(time_dens, dense_theory,
                                                                                           c=slowTheory, lw=1.8, ls='dotted', label='Theory')
                                                                            #plt.plot(time_dens, dense_n_arr * (np.pi/4),
                                                                            #               c=fastTheory, lw=1.8, ls='--', label='Simulation')
                                                                            plt.plot(time_width, dens_sim2,
                                                                                           c=fastTheory, lw=1.8, ls='--', label='Simulation')
                                                                            ax1.set_xlim(0, 200)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Area Fraction ($\phi$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=20)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=10)
                                                                            ax1.xaxis.set_minor_locator(loc)


                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=step)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=step/2)
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            # Left middle plot
                                                                            plt.legend(loc='upper right', fontsize=fsize*2.0)

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.savefig('/Volumes/External/dens_time2/slow_out_pa' + str(int(params['peA'][i])) + '_pb'+str(int(params['peB'][i])), dpi=100)
                                                                            plt.close()
                                                                            
                                                                            slowSim = '#de2d26'
                                                                            slowTheory = '#fc9272'
                                                                            fastSim = '#3182bd'
                                                                            fastTheory = '#9ecae1'
                                                                            
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))
                                                                            interpart_max = np.max(bulk_press_time[nonzero_bulk][10:])
                                                                            align_max = np.max(final_align_press[nonzero_align][10:])
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 500
                                                                            else:
                                                                                plot_max = interpart_max + 500

                                                                            interpart_min = np.min(bulk_press_time[nonzero_bulk][10:])
                                                                            align_min = np.min(final_align_press[nonzero_align][10:])
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 500
                                                                            else:
                                                                                plot_min = interpart_min - 500
                                                                            step = np.floor((plot_max - plot_min)/6)


                                                                            plt.plot(bulk_time[nonzero_bulk], bulk_press_time[nonzero_bulk],
                                                                                           c=slowSim, lw=1.8, ls='--', label='Sim Bulk')
                                                                            plt.plot(time_i[nonzero_align], final_align_press[nonzero_align],
                                                                                           c=fastSim, lw=1.8, ls='--', label='Sim Interface')
                                                                            plt.plot(time_dens, theoryss,
                                                                                           c=slowTheory, lw=1.8, ls='dotted', label='Theory Bulk')
                                                                            plt.plot(press_theory_time, press_theory_val,
                                                                                           c=fastTheory, lw=1.8, ls='dotted', label='Theory Interface')
                                                                            ax1.set_xlim(0, 200)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=20)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=10)
                                                                            ax1.xaxis.set_minor_locator(loc)


                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=step)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=int(step/2))
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            
                                                                            fast_leg=[]
                                                                            mkSz = [0.8, 0.8]
                                                                            msz=10
                                                                            #     eps_leg.append(Line2D([0], [0], lw=0., marker=mk, markeredgewidth=med,
                                                                            #                           markeredgecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
                                                                            #                           markerfacecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
                                                                            #                           label=r'$10^{{{}}}$'.format(-i), markersize=msz))
                                                                            fast_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=1.8*1.2,
                                                                                                  markeredgecolor=fastTheory,
                                                                                                  markerfacecolor=fastTheory, 
                                                                                                  label='Interface', markersize=(2.4*msz * mkSz[0])))
                                                                            fast_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=1.8*1.2,
                                                                                                  markeredgecolor=slowTheory,
                                                                                                  markerfacecolor=slowTheory, 
                                                                                                  label='Bulk', markersize=(2.4*msz * mkSz[0])))
                                                                            plt.text(0.44, 0.9, s='Interface:',
                                                                                    fontsize=fsize*2.4, transform = ax1.transAxes)
                                                                            plt.text(0.4925, 0.8025, s='Bulk:',
                                                                                    fontsize=fsize*2.4, transform = ax1.transAxes)
                                                                            leg_three = [Line2D([0], [0], lw=1.8*1.8, c=fastSim, markeredgecolor='none', label=r'Simulation', markerfacecolor='none', markersize=msz, ls='dashed'), Line2D([0], [0], lw=1.8*1.8, c=slowSim, markeredgecolor='none', label=r'Simulation', markerfacecolor='none', markersize=msz, ls='dashed'), Line2D([0], [0], lw=1.8*1.8, c=fastTheory, markeredgecolor='none', label=r'Theory', markerfacecolor='none', markersize=msz, ls='dotted'), Line2D([0], [0], lw=1.8*1.8, c=slowTheory, markeredgecolor='none', label=r'Theory', markerfacecolor='none', markersize=msz, ls='dotted')]
                                                                            two_leg = ax1.legend(handles=leg_three, loc='upper right',
                                                                                         columnspacing=1., handletextpad=0.1,
                                                                                         bbox_transform=ax1.transAxes, bbox_to_anchor=[0.98, 1.03],
                                                                                         fontsize=fsize*2.2, frameon=False, ncol=2)
                                                                            ax1.add_artist(two_leg)
                                                                            
                                                                            # Left middle plot

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.savefig('/Volumes/External/pressure_time2/slow_out_pa' + str(int(params['peA'][i])) + '_pb'+str(int(params['peB'][i])), dpi=100)
                                                                            plt.close()
                                                                            
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))
                                                                            interpart_max = np.max(bulk_press_time[nonzero_bulk])
                                                                            align_max = np.max(final_align_press[nonzero_align])
                                                                            
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 200
                                                                            else:
                                                                                plot_max = interpart_max + 200

                                                                            interpart_min = np.min(bulk_press_time[nonzero_bulk])
                                                                            align_min = np.min(final_align_press[nonzero_align])
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 200
                                                                            else:
                                                                                plot_min = interpart_min - 200
                                                                                
                                                                            step = np.floor((plot_max - plot_min)/6)
                                                                            
                                                                            
                                                                            
                                                                            
                                                                            
                                                                            plt.plot(bulk_time[nonzero_bulk], bulk_press_time[nonzero_bulk],
                                                                                           c=slowSim, lw=1.8*1.8, ls='--', label='Sim Bulk')
                                                                            plt.plot(time_i[nonzero_align], final_align_press[nonzero_align],
                                                                                           c=fastSim, lw=1.8*1.8, ls='--', label='Sim Interface')
                                                                            plt.plot(time_dens, theoryss,
                                                                                           c=slowTheory, lw=1.8*1.8, ls='dotted', label='Theory Bulk')
                                                                            plt.plot(press_theory_time, press_theory_val,
                                                                                           c=fastTheory, lw=1.8*1.8, ls='dotted', label='Theory Interface')
                                                                            ax1.set_xlim(0, 10)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=1)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=0.5)
                                                                            ax1.xaxis.set_minor_locator(loc)
                                                                            fast_leg=[]
                                                                            mkSz = [0.8, 0.8]
                                                                            msz=10
                                                                            #     eps_leg.append(Line2D([0], [0], lw=0., marker=mk, markeredgewidth=med,
                                                                            #                           markeredgecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
                                                                            #                           markerfacecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
                                                                            #                           label=r'$10^{{{}}}$'.format(-i), markersize=msz))
                                                                            fast_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=1.8*1.2,
                                                                                                  markeredgecolor=fastTheory,
                                                                                                  markerfacecolor=fastTheory, 
                                                                                                  label='Interface', markersize=(2.4*msz * mkSz[0])))
                                                                            fast_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=1.8*1.2,
                                                                                                  markeredgecolor=slowTheory,
                                                                                                  markerfacecolor=slowTheory, 
                                                                                                  label='Bulk', markersize=(2.4*msz * mkSz[0])))
                                                                            plt.text(0.44, 0.9, s='Interface:',
                                                                                    fontsize=fsize*2.4, transform = ax1.transAxes)
                                                                            plt.text(0.4925, 0.8025, s='Bulk:',
                                                                                    fontsize=fsize*2.4, transform = ax1.transAxes)
                                                                            leg_three = [Line2D([0], [0], lw=1.8*1.8, c=fastSim, markeredgecolor='none', label=r'Simulation', markerfacecolor='none', markersize=msz, ls='dashed'), Line2D([0], [0], lw=1.8*1.8, c=slowSim, markeredgecolor='none', label=r'Simulation', markerfacecolor='none', markersize=msz, ls='dashed'), Line2D([0], [0], lw=1.8*1.8, c=fastTheory, markeredgecolor='none', label=r'Theory', markerfacecolor='none', markersize=msz, ls='dotted'), Line2D([0], [0], lw=1.8*1.8, c=slowTheory, markeredgecolor='none', label=r'Theory', markerfacecolor='none', markersize=msz, ls='dotted')]
                                                                            two_leg = ax1.legend(handles=leg_three, loc='upper right',
                                                                                         columnspacing=1., handletextpad=0.1,
                                                                                         bbox_transform=ax1.transAxes, bbox_to_anchor=[0.98, 1.03],
                                                                                         fontsize=fsize*2.2, frameon=False, ncol=2)
                                                                            ax1.add_artist(two_leg)
                                                                            
                                                                            
                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=step)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=int(step/2))
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            # Left middle plot
                                                                            #plt.legend(loc='upper right', fontsize=fsize*2.0)

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.savefig('/Volumes/External/pressure_time2/slow_out_zoom_pa' + str(int(params['peA'][i])) + '_pb'+str(int(params['peB'][i])), dpi=100)
                                                                            plt.close()
                                                                            
                                                                            avg_bulk_theory_arr = np.append(avg_bulk_theory_arr, np.mean(theoryss))
                                                                            avg_int_theory_arr = np.append(avg_int_theory_arr, np.mean(press_theory_val))














        '''

                                                                    print(len(xA_arr_final))
                                                                    print(len(press_arr))
                                                                    print(len(time_id2))
                                                                    plt.plot(press_theory_time, press_theory_val)
                                                                    plt.show()
                                                                    plt.plot(radius_arr_final, xA_arr_temp, color='blue')
                                                                    plt.plot(radius_arr_final, xB_arr_temp, color='red')
                                                                    plt.plot(radius_arr_final, num_dens_arr_temp, color='green')
                                                                    plt.show()
                                                                    stop
                                                                    print(params['peA'][i])
                                                                    print(params['peB'][i])
                                                                    print(paramss['peA'][m])
                                                                    print(paramss['peB'][m])

                                                                    peNet_int = (((int_nA_arr/int_n_arr) * float(params['peA'][i]) + (int_nB_arr/int_n_arr) * float(params['peB'][i])))
                                                                    peNet_ints = (((int_nA_arrs/int_n_arrs) * float(paramss['peA'][m]) + (int_nB_arrs/int_n_arrs) * float(paramss['peB'][m])))

                                                                    peNet = params2['peB'][k] * (1-params2['xA'][k]/100) + params2['peA'][k] * (params2['xA'][k]/100)
                                                                    latNet_int = np.zeros(len(peNet_int))
                                                                    for z in range(0, len(peNet_int)):
                                                                        latNet_int[z] = getLat(peNet_int[z] - 50, 1.0)
                                                                    latNet_ints = np.zeros(len(peNet_ints))
                                                                    for z in range(0, len(peNet_ints)):
                                                                        latNet_ints[z] = getLat(peNet_ints[z] - 50, 1.0)

                                                                    theoryss = (peNet_int-50) * 2 * 1.92 * np.sqrt(3) / latNet_int
                                                                    theorysss = (peNet_ints-50) * 2 * 1.92 * np.sqrt(3) / latNet_ints


                                                                    x_range2 = np.array([10000000, 20000000])
                                                                    y_range2 = np.array([theoryss, theoryss])

                                                                    x_range_int = np.array([0.3, 0.3+0.0000000000001])
                                                                    y_range_int = np.array([-1000000, 1000000])

                                                                    fsize=10
                                                                    mpl.rcParams.update({'font.size': 13})
                                                                    mkSz = [0.1, 0.1, 0.15, 0.1, 0.1]
                                                                    msz=40
                                                                    yellow = ("#fec44f")
                                                                    green = ("#77dd77")
                                                                    red = ("#ff6961")


                                                                    nonzero_bulk = np.where(bulk_press_time>0)[0]
                                                                    nonzero_align = np.where(align_press_time>0)[0]
                                                                    nonzero_bulks = np.where(bulk_press_times>0)[0]
                                                                    nonzero_aligns = np.where(align_press_times>0)[0]
                                                                    if len(nonzero_bulk)>10:
                                                                        if len(nonzero_align)>10:
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))
                                                                            interpart_max = np.max(bulk_press_time[nonzero_bulk][10:])
                                                                            align_max = np.max(align_press_time[nonzero_align][10:])
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 2000
                                                                            else:
                                                                                plot_max = interpart_max + 2000

                                                                            interpart_min = np.min(bulk_press_time[nonzero_bulk][10:])
                                                                            align_min = np.min(align_press_time[nonzero_align][10:])
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 2000
                                                                            else:
                                                                                plot_min = interpart_min - 2000


                                                                            plt.plot(bulk_time[nonzero_bulk], bulk_press_time[nonzero_bulk],
                                                                                           c=slowCol, lw=1.8*1.8, ls='--')
                                                                            plt.plot(align_time[nonzero_align], align_press_time[nonzero_align],
                                                                                           c=fastCol, lw=1.8*1.8, ls='--')
                                                                            plt.plot(time_arr3, theoryss,
                                                                                           c=green, lw=1.8*1.8, ls='-', label='Theory')
                                                                            ax1.set_xlim(0, 200)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=20)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=10)
                                                                            ax1.xaxis.set_minor_locator(loc)


                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=1000)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=500)
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            # Left middle plot
                                                                            plt.legend(loc='upper right', fontsize=fsize*2.7)

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.show()






                                                                    nonzero_bulk = np.where(bulk_press_time>0)[0]
                                                                    nonzero_align = np.where(align_press_time>0)[0]
                                                                    nonzero_bulks = np.where(bulk_press_times>0)[0]
                                                                    nonzero_aligns = np.where(align_press_times>0)[0]
                                                                    if len(nonzero_bulks)>10:
                                                                        if len(nonzero_aligns)>10:
                                                                            fig, ax1 = plt.subplots(figsize=(12,5))
                                                                            interpart_max = np.max(bulk_press_times[nonzero_bulks][10:])
                                                                            align_max = np.max(align_press_times[nonzero_aligns][10:])
                                                                            if align_max >= interpart_max:
                                                                                plot_max = align_max + 2000
                                                                            else:
                                                                                plot_max = interpart_max + 2000

                                                                            interpart_min = np.min(bulk_press_times[nonzero_bulks][10:])
                                                                            align_min = np.min(align_press_times[nonzero_aligns][10:])
                                                                            if align_min <= interpart_min:
                                                                                plot_min = align_min - 2000
                                                                            else:
                                                                                plot_min = interpart_min - 2000

                                                                            plt.plot(bulk_times[nonzero_bulks], bulk_press_times[nonzero_bulks],
                                                                                           c=fastCol, lw=1.8*1.8, ls='-', label='Bulk')
                                                                            plt.plot(align_times[nonzero_aligns], align_press_times[nonzero_aligns],
                                                                                           c=slowCol, lw=1.8*1.8, ls='-', label='Interface')
                                                                            plt.plot(time_arr3s, theorysss,
                                                                                           c=green, lw=1.8*1.8, ls='-', label='Theory')

                                                                            ax1.set_xlim(0, 200)        
                                                                            # y limits


                                                                            ax1.set_ylim(plot_min, plot_max) 


                                                                            ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                            ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.8)


                                                                            # Set all the x ticks for radial plots
                                                                            loc = ticker.MultipleLocator(base=20)
                                                                            ax1.xaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=10)
                                                                            ax1.xaxis.set_minor_locator(loc)


                                                                            # Set y ticks
                                                                            loc = ticker.MultipleLocator(base=1000)
                                                                            ax1.yaxis.set_major_locator(loc)
                                                                            loc = ticker.MultipleLocator(base=500)
                                                                            ax1.yaxis.set_minor_locator(loc)
                                                                            # Left middle plot
                                                                            plt.legend(loc='upper right', fontsize=fsize*2.7)

                                                                            ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                            ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                            plt.tight_layout()
                                                                            plt.show()



        '''


100
100
150
100
150
150
200
100
200
150
200
200
200
50
250
50
250
100
250
150
250
200


<ipython-input-25-dd896da63b50>:365: RuntimeWarning: invalid value encountered in double_scalars
  xA_arr = np.append(xA_arr, all_pres_new3[s]['num_densA'][y] / all_pres_new3[s]['num_dens'][y])
<ipython-input-25-dd896da63b50>:366: RuntimeWarning: invalid value encountered in double_scalars
  xB_arr = np.append(xB_arr, all_pres_new3[s]['num_densB'][y] / all_pres_new3[s]['num_dens'][y])
<ipython-input-25-dd896da63b50>:365: RuntimeWarning: invalid value encountered in double_scalars
  xA_arr = np.append(xA_arr, all_pres_new3[s]['num_densA'][y] / all_pres_new3[s]['num_dens'][y])
<ipython-input-25-dd896da63b50>:366: RuntimeWarning: invalid value encountered in double_scalars
  xB_arr = np.append(xB_arr, all_pres_new3[s]['num_densB'][y] / all_pres_new3[s]['num_dens'][y])


250
250
350
100


<ipython-input-25-dd896da63b50>:365: RuntimeWarning: invalid value encountered in double_scalars
  xA_arr = np.append(xA_arr, all_pres_new3[s]['num_densA'][y] / all_pres_new3[s]['num_dens'][y])
<ipython-input-25-dd896da63b50>:366: RuntimeWarning: invalid value encountered in double_scalars
  xB_arr = np.append(xB_arr, all_pres_new3[s]['num_densB'][y] / all_pres_new3[s]['num_dens'][y])


350
200
350
250
350
350
350
50
450
450
450
50
450
100
450
200
450
250
450
350


In [ ]:
import matplotlib.patches as mpatches
from matplotlib import cm
import matplotlib.patches as patches

lat_theory_mean_int = np.zeros(len(interpart_peNet))
lat_theory_mean_bulk = np.zeros(len(interpart_peNet))

lat_theory_mean_dense = np.zeros(len(interpart_peNet))

gas_pe_net = np.zeros(len(interpart_peNet))

dense_pe_net = np.zeros(len(interpart_peNet))
bulk_pe_net = np.zeros(len(interpart_peNet))
int_pe_net = np.zeros(len(interpart_peNet))
pe_dif_arr = np.zeros(len(interpart_peNet))

beta_theory = np.zeros(len(interpart_peNet))
beta_theory2 = np.zeros(len(interpart_peNet))

beta_theory3 = np.zeros(len(interpart_peNet))
beta_theory4 = np.zeros(len(interpart_peNet))

phi_gas_sim = np.zeros(len(interpart_peNet))



avg_area2 = np.zeros(len(interpart_peNet))

div = (1.0/bulk_lat_mean)
dU = (24.) * ((2*(div**13)) - (div)**7)
phi_cp = np.pi / (2*np.sqrt(3))
area_box = (50000 * (np.pi/4))/0.6


int_pe_net_theory = np.linspace(50, 500, num=451)

phi_gas_theory2 = np.zeros(len(int_pe_net_theory))
phi_dense_theory2 = np.zeros(len(int_pe_net_theory))
clust_theory2 = np.zeros(len(int_pe_net_theory))
rad_theory2 = np.zeros(len(int_pe_net_theory))
press_theory2 = np.zeros(len(int_pe_net_theory))
width_theory2 = np.zeros(len(int_pe_net_theory))

lat_theory_mean_int_theory = np.zeros(len(int_pe_net_theory))


phi_gas_theory3 = np.zeros(len(interpart_peNet))
phi_gas_theory4 = np.zeros(len(interpart_peNet))
phi_gas_theory5 = np.zeros(len(interpart_peNet))

phi_gas_theory = np.zeros(len(interpart_peNet))
phi_dense_theory = np.zeros(len(interpart_peNet))
clust_theory = np.zeros(len(interpart_peNet))
rad_theory = np.zeros(len(interpart_peNet))
press_theory = np.zeros(len(interpart_peNet))
#lat_theory_mean_int_theory = np.zeros(len(interpart_peNet))

for p in range(0, len(interpart_peNet)):
    gas_pe_net[p]=(((avg_gas_nA[p]/avg_gas_n[p]) * interpart_peA[p] + (avg_gas_nB[p]/avg_gas_n[p]) * interpart_peB[p]))
    dense_pe_net[p]=(((avg_dense_nA[p]/avg_dense_n[p]) * interpart_peA[p] + (avg_dense_nB[p]/avg_dense_n[p]) * interpart_peB[p]))
    bulk_pe_net[p]=((avg_bulk_nA[p]/avg_bulk_n[p]) * interpart_peA[p] + (avg_bulk_nB[p]/avg_bulk_n[p]) * interpart_peB[p])
    int_pe_net[p]=(((avg_int_nA[p]/avg_int_n[p]) * interpart_peA[p] + (avg_int_nB[p]/avg_int_n[p]) * interpart_peB[p]))
    pe_dif_arr[p] = (interpart_peB[p] - interpart_peA[p])
    
    beta_theory[p] = dU[p]/interpart_peB[p]
    #print(beta_theory[p])
    beta_theory2[p] = dU[p]/bulk_pe_net[p]
    beta_theory3[p] = dU[p]/dense_pe_net[p]
    beta_theory4[p] = dU[p]/interpart_peNet[p]
    lat_theory_mean_int[p] = conForRClust(int_pe_net[p]-50, 1.0)
    lat_theory_mean_bulk[p] = conForRClust(bulk_pe_net[p]-50, 1.0)
    lat_theory_mean_dense[p] = conForRClust(dense_pe_net[p]-50, 1.0)
    phi_gas_sim[p] = ((50000-avg_clust[p]) * (np.pi/4)) /(area_box - np.pi * avg_rad[p]**2)
    phi_dense_theory[p] = phi_cp / lat_theory_mean_int[p]**2 
    phi_gas_theory[p] = (3 * np.pi**2 * 4.5/4) * ((lat_theory_mean_int[p] * int_pe_net[p]) ** (-1))
    phi_gas_theory3[p] = (3 * np.pi**2 * 4.5/4) * ((lat_theory_mean_int[p] * dense_pe_net[p]) ** (-1))
    phi_gas_theory4[p] = (3 * np.pi**2 * 4.5/4) * ((lat_theory_mean_int[p] * gas_pe_net[p]) ** (-1))

    phi_gas_theory5[p] = (3 * np.pi**2 * 4.5/4) * ((lat_theory_mean_int[p] * bulk_pe_net[p]) ** (-1))
    
    clust_theory[p] = 50000* (((phi_gas_theory[p]-0.6)*phi_dense_theory[p])/(0.6*(phi_gas_theory[p]-phi_dense_theory[p])))
    avg_area2[p] = avg_clust[p] / (np.pi * avg_rad[p]**2)
    press_theory[p] = (int_pe_net[p]-50) * 2 * np.sqrt(3) * 1.874 /lat_theory_mean_int[p]
    rad_theory[p] =  ((avg_clust[p] / (4*phi_cp))**0.5) * lat_theory_mean_int[p]
for p in range(0, len(int_pe_net_theory)):
    lat_theory_mean_int_theory[p] = conForRClust(int_pe_net_theory[p]-50, 1.0)
    phi_dense_theory2[p] = phi_cp / lat_theory_mean_int_theory[p]**2 
    phi_gas_theory2[p] = (3 * np.pi**2 * 4.5/4) * ((lat_theory_mean_int_theory[p] * int_pe_net_theory[p]) ** (-1))

    clust_theory2[p] = 50000* (((phi_gas_theory2[p]-0.6)*phi_dense_theory2[p])/(0.6*(phi_gas_theory2[p]-phi_dense_theory2[p])))
    press_theory2[p] = (int_pe_net_theory[p]-50) * 2 * np.sqrt(3) * 1.874 /lat_theory_mean_int_theory[p]
    rad_theory2[p] =  ((clust_theory2[p] / (4*phi_cp))**0.5) * lat_theory_mean_int_theory[p]
    width_theory2[p] = (np.sqrt(3)/(2*0.5)) * (press_theory2[p]/int_pe_net_theory[p]) * lat_theory_mean_int_theory[p]**2 * 3.0
radius_arr_temp = np.linspace(0, 1.6, num=60)
int_pe_net_theory_min_arr = np.array([])
int_pe_net_theory_max_arr = np.array([])

press_theory_val_min = np.array([])
press_theory_val_max = np.array([])

int_pe_net_theory2 = np.linspace(50, 500, num=91)
press_theory_val_arr = np.array([])

int_pe_net_theory_arr = np.array([])


for x in range(0, len(int_pe_net_theory2)):
    print(x)
    for y in range(0, len(int_pe_net_theory2)):
        if int_pe_net_theory2[x]<= int_pe_net_theory2[y]:
            int_pe_net_theory_temp = np.linspace(int(int_pe_net_theory2[x]), int(int_pe_net_theory2[y]), num = int((int_pe_net_theory2[y]-int_pe_net_theory2[x])+1))
            
            for z in range(0, len(int_pe_net_theory_temp)):
                
                lat_theory_mean_int_val = conForRClust(int_pe_net_theory_temp[z]-50, 1.0)
                phi_dense_val = phi_cp / lat_theory_mean_int_val**2 
                phi_gas_val = (3 * np.pi**2 * 4.5/4) * ((lat_theory_mean_int_val * int_pe_net_theory_temp[z]) ** (-1))

                clust_val = 50000* (((phi_gas_val-0.6)*phi_dense_val)/(0.6*(phi_gas_val-phi_dense_val)))
                press_val = (int_pe_net_theory_temp[z]-50) * 2 * np.sqrt(3) * 1.874 /lat_theory_mean_int_val
                rad_val =  ((clust_val / (4*phi_cp))**0.5) * lat_theory_mean_int_val
                width_val = (np.sqrt(3)/(2*0.5)) * (press_val/int_pe_net_theory_temp[z]) * lat_theory_mean_int_val**2 * 3.0
                peNet = 0.5 * float(int_pe_net_theory2[x]) + 0.5 * float(int_pe_net_theory2[y])
                #int_pe_net_val = 

                dens = densProbability(radius_arr_temp, peNet, int_pe_net_theory2[x])
                align = alignProbability(radius_arr_temp, peNet, int_pe_net_theory2[x])

                press = dens * align * int_pe_net_theory_temp[z]
                press_int = 0
                for m in range(1, len(press)):
                    press_int += ((press[m-1]+press[m])/2)*(radius_arr_temp[m]-radius_arr_temp[m-1]) * rad_val
                press_theory_val_arr = np.append(press_theory_val_arr, press_int)
                int_pe_net_theory_arr = np.append(int_pe_net_theory_arr, int_pe_net_theory_temp[z])

int_pe_net_final_arr = np.array([])
press_theory_max_arr = np.array([])
press_theory_min_arr = np.array([])
for z in range(0, len(int_pe_net_theory_arr)):
    if int_pe_net_theory_arr[z] not in int_pe_net_final_arr:
        loc = np.where(int_pe_net_theory_arr==int_pe_net_theory_arr[z])[0]
        press_theory_max_arr = np.append(press_theory_max_arr, np.max(press_theory_val_arr[loc]))
        press_theory_min_arr = np.append(press_theory_min_arr, np.min(press_theory_val_arr[loc]))
        int_pe_net_final_arr = np.append(int_pe_net_final_arr, int_pe_net_theory_arr[z])


In [ ]:
yellow = ("#fdfd96")
green = ("#77dd77")
red = ("#ff6961")
purple = ("#cab2d6")

msz=20
# With fill
mk='s'
med = 1.5
mkSz = [0.8, 0.8]
msz=10

fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
plt.plot(int_pe_net_theory, clust_theory2, color='black', linestyle='dashed', linewidth=1.8*1.8)

plt.scatter(int_pe_net, avg_clust, edgecolor=green,facecolor=green,  s=msz*55*0.7)
#plt.scatter(int_pe_net, phi_dense_theory, edgecolor=green, facecolor='none', s=25.0, label='Dense Theory')
#plt.plot(int_pe_net_theory, phi_dense_theory2, color='black', linestyle='--', linewidth=1.8*1.8)

#plt.scatter(int_pe_net, avg_area2 * (np.pi/4), edgecolor=green, facecolor=green, s=msz*55*0.7)

ax1.set_xlim([75.0,500.0])
ax1.set_ylim([0, 50000])

ax1.tick_params(axis='x', labelsize=fsize*1.9)
ax1.tick_params(axis='y', labelsize=fsize*1.9)
ax1.set_ylabel(r'Cluster Size ($N_\mathrm{d}$)', fontsize=fsize*2.1)
ax1.set_xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)', fontsize=fsize*2.1)




leg_three = [Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 13', markerfacecolor='none', markersize=msz, ls='dashed')]

two_leg = ax1.legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.3, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=1)
ax1.add_artist(two_leg)



plt.show()




fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
plt.plot(int_pe_net_theory, width_theory2, color='black', linestyle='dashed', linewidth=1.8*1.8)

plt.scatter(int_pe_net, avg_width, edgecolor=green,facecolor=green,  s=msz*55*0.7)
#plt.scatter(int_pe_net, phi_dense_theory, edgecolor=green, facecolor='none', s=25.0, label='Dense Theory')
#plt.plot(int_pe_net_theory, phi_dense_theory2, color='black', linestyle='--', linewidth=1.8*1.8)

#plt.scatter(int_pe_net, avg_area2 * (np.pi/4), edgecolor=green, facecolor=green, s=msz*55*0.7)

ax1.set_xlim([75.0,500.0])
ax1.set_ylim([0, 30])

ax1.tick_params(axis='x', labelsize=fsize*1.9)
ax1.tick_params(axis='y', labelsize=fsize*1.9)
ax1.set_ylabel(r'Interface Width ($h$)', fontsize=fsize*2.1)
ax1.set_xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)', fontsize=fsize*2.1)

msz=20
# With fill
mk='s'
med = 1.5
mkSz = [0.8, 0.8]
msz=10


leg_three = [Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 13', markerfacecolor='none', markersize=msz, ls='dashed')]

two_leg = ax1.legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.3, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=1)
ax1.add_artist(two_leg)



plt.show()

fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
plt.plot(int_pe_net_theory, width_theory2/lat_theory_mean_int_theory, color='black', linestyle='dashed', linewidth=1.8*1.8)

plt.scatter(int_pe_net, avg_width/all_lat_time_mean, edgecolor=green,facecolor=green,  s=msz*55*0.7)
#plt.scatter(int_pe_net, phi_dense_theory, edgecolor=green, facecolor='none', s=25.0, label='Dense Theory')
#plt.plot(int_pe_net_theory, phi_dense_theory2, color='black', linestyle='--', linewidth=1.8*1.8)

#plt.scatter(int_pe_net, avg_area2 * (np.pi/4), edgecolor=green, facecolor=green, s=msz*55*0.7)

ax1.set_xlim([75.0,500.0])
ax1.set_ylim([0, 30])

ax1.tick_params(axis='x', labelsize=fsize*1.9)
ax1.tick_params(axis='y', labelsize=fsize*1.9)
ax1.set_ylabel(r'Interface Width:Lattice Spacing ($h/a$)', fontsize=fsize*2.1)
ax1.set_xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)', fontsize=fsize*2.1)

msz=20
# With fill
mk='s'
med = 1.5
mkSz = [0.8, 0.8]
msz=10


leg_three = [Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 13', markerfacecolor='none', markersize=msz, ls='dashed')]

two_leg = ax1.legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.3, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=1)
ax1.add_artist(two_leg)



plt.show()

fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
plt.plot(int_pe_net_theory, lat_theory_mean_int_theory, color='black', linestyle='dashed', linewidth=1.8*1.8)

plt.scatter(int_pe_net, all_lat_time_mean, edgecolor=green,facecolor=green,  s=msz*55*0.7)
#plt.scatter(int_pe_net, phi_dense_theory, edgecolor=green, facecolor='none', s=25.0, label='Dense Theory')
#plt.plot(int_pe_net_theory, phi_dense_theory2, color='black', linestyle='--', linewidth=1.8*1.8)

#plt.scatter(int_pe_net, avg_area2 * (np.pi/4), edgecolor=green, facecolor=green, s=msz*55*0.7)

ax1.set_xlim([75.0,500.0])
ax1.set_ylim([0, 1.2])

ax1.tick_params(axis='x', labelsize=fsize*1.9)
ax1.tick_params(axis='y', labelsize=fsize*1.9)
ax1.set_ylabel(r'Lattice Spacing ($a$)', fontsize=fsize*2.1)
ax1.set_xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)', fontsize=fsize*2.1)

msz=20
# With fill
mk='s'
med = 1.5
mkSz = [0.8, 0.8]
msz=10


leg_three = [Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 13', markerfacecolor='none', markersize=msz, ls='dashed')]

two_leg = ax1.legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.3, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=1)
ax1.add_artist(two_leg)



plt.show()

fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
plt.plot(int_pe_net_theory, rad_theory2, color='black', linestyle='dashed', linewidth=1.8*1.8)

plt.scatter(int_pe_net, avg_rad, edgecolor=green,facecolor=green,  s=msz*55*0.7)
#plt.scatter(int_pe_net, phi_dense_theory, edgecolor=green, facecolor='none', s=25.0, label='Dense Theory')
#plt.plot(int_pe_net_theory, phi_dense_theory2, color='black', linestyle='--', linewidth=1.8*1.8)

#plt.scatter(int_pe_net, avg_area2 * (np.pi/4), edgecolor=green, facecolor=green, s=msz*55*0.7)

ax1.set_xlim([75.0,500.0])
ax1.set_ylim([0, 120])

ax1.tick_params(axis='x', labelsize=fsize*1.9)
ax1.tick_params(axis='y', labelsize=fsize*1.9)
ax1.set_ylabel(r'Cluster Radius ($r_\mathrm{c}$)', fontsize=fsize*2.1)
ax1.set_xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)', fontsize=fsize*2.1)

msz=20
# With fill
mk='s'
med = 1.5
mkSz = [0.8, 0.8]
msz=10


leg_three = [Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 15', markerfacecolor='none', markersize=msz, ls='dashed')]

two_leg = ax1.legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.3, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=1)
ax1.add_artist(two_leg)



plt.show()
    
fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
plt.plot(int_pe_net_theory, phi_gas_theory2, color='black', linestyle='dotted', linewidth=1.8*1.8)

plt.scatter(int_pe_net, phi_gas_sim, edgecolor=red,facecolor=red,  s=msz*55*0.7, label='Gas Sim')
#plt.scatter(int_pe_net, phi_dense_theory, edgecolor=green, facecolor='none', s=25.0, label='Dense Theory')
plt.plot(int_pe_net_theory, phi_dense_theory2, color='black', linestyle='--', linewidth=1.8*1.8)

plt.scatter(int_pe_net, avg_area2 * (np.pi/4), edgecolor=green, facecolor=green, s=msz*55*0.7, label='Dense Sim')

ax1.set_xlim([75.0,500.0])
ax1.set_ylim([0, 1.6])

ax1.tick_params(axis='x', labelsize=fsize*1.9)
ax1.tick_params(axis='y', labelsize=fsize*1.9)
ax1.set_ylabel(r'Avg Area Fraction ($\phi$)', fontsize=fsize*2.1)
ax1.set_xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)', fontsize=fsize*2.1)

msz=20
# With fill
mk='s'
med = 1.5
eps_leg=[]
mkSz = [0.8, 0.8]
msz=10
#     eps_leg.append(Line2D([0], [0], lw=0., marker=mk, markeredgewidth=med,
#                           markeredgecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
#                           markerfacecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
#                           label=r'$10^{{{}}}$'.format(-i), markersize=msz))
eps_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=med,
                      markeredgecolor=green,
                      markerfacecolor=green,
                      label='Dense', markersize=(2.4*msz * mkSz[0])))
eps_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=med,
                      markeredgecolor=red,
                      markerfacecolor=red,
                      label='Gas', markersize=(2.4*msz * mkSz[0])))

leg_three = [Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 8', markerfacecolor='none', markersize=msz, ls='dashed'),
             Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 12', markerfacecolor='none', markersize=msz, ls='dotted')]

two_leg = ax1.legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.3, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=1)
ax1.add_artist(two_leg)

one_leg = ax1.legend(handles=eps_leg, loc='center',
             columnspacing=0.08, handletextpad=-0.5,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.8, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=5)
ax1.add_artist(one_leg)


plt.show()

fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')

plt.scatter(int_pe_net, 100*(phi_gas_sim-phi_gas_theory)/phi_gas_theory, edgecolor=red,facecolor=red,  s=25.0, label='Gas Sim')

plt.scatter(int_pe_net, 100*(avg_area2 * (np.pi/4) - phi_dense_theory)/phi_dense_theory, edgecolor=green, facecolor=green, s=25.0, label='Dense Sim')

ax1.set_xlim([0,500.0])
ax1.set_ylim([-100, 100])
plt.ylabel(r'Avg Area Fraction ($\phi$)')
plt.xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)')
plt.legend(loc = 'upper right')

plt.show()

error = 100*(phi_gas_sim-phi_gas_theory)/phi_gas_theory
avg_error = np.mean(error)
sumi=0
for i in range(0, len(error)):
    sumi+= (error[i]-avg_error)**2
std = (sumi/len(error))**0.5
print(avg_error)
print(std)

error = 100*(phi_gas_sim-phi_gas_theory3)/phi_gas_theory3
avg_error = np.mean(error)
sumi=0
for i in range(0, len(error)):
    sumi+= (error[i]-avg_error)**2
std = (sumi/len(error))**0.5
print(avg_error)
print(std)

error = 100*(phi_gas_sim-phi_gas_theory4)/phi_gas_theory4
avg_error = np.mean(error)
sumi=0
for i in range(0, len(error)):
    sumi+= (error[i]-avg_error)**2
std = (sumi/len(error))**0.5
print(avg_error)
print(std)

error = 100*(phi_gas_sim-phi_gas_theory5)/phi_gas_theory5
avg_error = np.mean(error)
sumi=0
for i in range(0, len(error)):
    sumi+= (error[i]-avg_error)**2
std = (sumi/len(error))**0.5
print(avg_error)
print(std)


fig, ax1 = plt.subplots(figsize=(8,7))
#plt.scatter(int_pe_net, phi_gas_theory2, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.scatter(int_pe_net, phi_gas_theory, edgecolor=red, facecolor='none', s=25.0, label='Gas Theory')
#plt.plot(int_pe_net, avg_int_theory_arr, color='black', linestyle='dotted', linewidth=1.8*1.8)
ax1.fill_between(int_pe_net_final_arr, press_theory_min_arr, press_theory_max_arr, color='black', alpha=0.2)

plt.scatter(int_pe_net, align_press_arr, edgecolor=yellow,facecolor=yellow,  s=msz*55*0.7, label='Gas Sim')
#plt.scatter(int_pe_net, phi_dense_theory, edgecolor=green, facecolor='none', s=25.0, label='Dense Theory')
plt.plot(int_pe_net_theory, press_theory2, color='black', linestyle='--', linewidth=1.8*1.8)
plt.plot(int_pe_net_final_arr, press_theory_min_arr, linestyle='dotted', color='black', linewidth=1.8*1.8)
plt.plot(int_pe_net_final_arr, press_theory_max_arr, linestyle='dotted', color='black', linewidth=1.8*1.8)
plt.scatter(int_pe_net, interpart_press, edgecolor=green, facecolor=green, s=msz*55*0.7, label='Dense Sim')

ax1.set_xlim([75.0,500.0])
ax1.set_ylim([0, 4000])

ax1.tick_params(axis='x', labelsize=fsize*1.9)
ax1.tick_params(axis='y', labelsize=fsize*1.9)
ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.1)
ax1.set_xlabel(r'Interface Net Activity ($\mathrm{Pe}_\mathrm{Net}^\mathrm{i}$)', fontsize=fsize*2.1)

msz=20
# With fill
mk='s'
med = 1.5
eps_leg=[]
mkSz = [0.8, 0.8]
msz=10
#     eps_leg.append(Line2D([0], [0], lw=0., marker=mk, markeredgewidth=med,
#                           markeredgecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
#                           markerfacecolor=plt.cm.jet((np.log10(myEps[i]) + 4)/ (len(myEps)-1) ),
#                           label=r'$10^{{{}}}$'.format(-i), markersize=msz))
eps_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=med,
                      markeredgecolor=green,
                      markerfacecolor=green,
                      label='Bulk', markersize=(2.4*msz * mkSz[0])))
eps_leg.append(Line2D([0], [0], lw=0., marker='o', markeredgewidth=med,
                      markeredgecolor=yellow,
                      markerfacecolor=yellow,
                      label='Interface', markersize=(2.4*msz * mkSz[0])))

leg_three = [Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 17', markerfacecolor='dashed', markersize=msz, ls='dashed'),
             Line2D([0], [0], lw=1.8*1.8, c='k', markeredgecolor='none', label=r'Eq. 26', markerfacecolor='dotted', markersize=msz, ls='dotted')]

two_leg = ax1.legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.3, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=1)
ax1.add_artist(two_leg)

one_leg = ax1.legend(handles=eps_leg, loc='center',
             columnspacing=0.08, handletextpad=-0.5,
             bbox_transform=ax1.transAxes, bbox_to_anchor=[0.75, 1.03],
             fontsize=fsize*2.2, frameon=False, ncol=5)
ax1.add_artist(one_leg)


plt.show()




In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(interpart_peNet, interpart_press, c='blue', label='bulk interparticle')
plt.scatter(align_peNet, align_press_arr, c='red', label='aligned interface')
plt.scatter(interpart_peNet, avg_int_theory_arr, c='yellow', label='theory int')
plt.scatter(interpart_peNet, avg_bulk_theory_arr, c='green', label='theory bulk')
plt.ylabel(r'Pressure ($\Pi$)')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}$')
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
plt.scatter(int_pe_net, interpart_press, c='blue', label='bulk interparticle')
plt.scatter(int_pe_net, align_press_arr, c='red', label='aligned interface')
plt.scatter(int_pe_net, avg_int_theory_arr, c='yellow', label='theory int')
plt.scatter(int_pe_net, avg_bulk_theory_arr, c='green', label='theory bulk')
plt.ylabel(r'Pressure ($\Pi$)')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}$')
plt.legend()
plt.show()

import matplotlib
fig = plt.figure(figsize=(8,6))
im = plt.scatter(int_pe_net, interpart_press, c=pe_dif_arr, cmap='viridis', vmin=np.min(pe_dif_arr), vmax=np.max(pe_dif_arr))
norm= matplotlib.colors.Normalize(vmin=np.min(pe_dif_arr), vmax=np.max(pe_dif_arr))

sm = plt.cm.ScalarMappable(norm=norm, cmap = im.cmap)
sm.set_array([])
clb = fig.colorbar(sm)#ticks=[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0], ax=ax2)

clb.ax.set_title(r'$\mathrm{Pe}_\mathrm{F}-\mathrm{Pe}_\mathrm{S}$', fontsize=15)
plt.xlim(0, 500)
plt.ylim(0, 3500)
plt.ylabel(r'Pressure ($\Pi_\mathrm{d}$)')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}^\mathrm{int}$')
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
im = plt.scatter(int_pe_net, align_press_arr, c=pe_dif_arr, cmap='viridis', vmin=np.min(pe_dif_arr), vmax=np.max(pe_dif_arr))
norm= matplotlib.colors.Normalize(vmin=np.min(pe_dif_arr), vmax=np.max(pe_dif_arr))
sm = plt.cm.ScalarMappable(norm=norm, cmap = im.cmap)
sm.set_array([])
clb = fig.colorbar(sm)#ticks=[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0], ax=ax2)

clb.ax.set_title(r'$\mathrm{Pe}_\mathrm{F}-\mathrm{Pe}_\mathrm{S}$', fontsize=15)
            
plt.xlim(0, 500)
plt.ylim(0, 3500)

#plt.ylim(0, l_box)
#plt.axis('off')
plt.ylabel(r'Pressure ($\Pi_\mathrm{int}$)')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}^\mathrm{int}$')
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
im = plt.scatter(int_pe_net, ((align_press_arr-interpart_press)/interpart_press)*100, c='black')
 
plt.xlim(0, 500)
plt.ylim(-100, 100)

#plt.ylim(0, l_box)
#plt.axis('off')
plt.ylabel(r'$100(\Pi_d - \Pi_i)/\Pi_i$')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}^\mathrm{int}$')

# Make a simulation vs theory legend
leg_three = [Line2D([0], [0], lw=med*1.8, c='k', markeredgecolor='none', label=r'Eq. 6', markerfacecolor='none', markersize=msz, ls='dashed'),
             Line2D([0], [0], lw=med*1.8, c='k', markeredgecolor='none', label=r'Eq. 7', markerfacecolor='none', markersize=msz, ls='dotted')]

two_leg = ax[0].legend(handles=leg_three, loc='upper right',
             columnspacing=1., handletextpad=0.1,
             bbox_transform=ax[0].transAxes, bbox_to_anchor=[1.0, 1.03],
             fontsize=fsize*3.2, frameon=False, ncol=1)
ax[0].add_artist(two_leg)

plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
plt.scatter(int_pe_net, ((interpart_press-avg_bulk_theory_arr)/avg_bulk_theory_arr)*100, c=green, label='bulk')
plt.scatter(int_pe_net, ((align_press_arr-avg_int_theory_arr)/avg_int_theory_arr)*100, c=yellow, label='interface')

plt.xlim(0, 500)
plt.ylim(-100, 100)

#plt.ylim(0, l_box)
#plt.axis('off')
plt.ylabel(r'$100(\Pi_\mathrm{sim} - \Pi_\mathrm{theory})/\Pi_\mathrm{theory}$')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}^\mathrm{int}$')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()